# Processing growth data from *Escherichia coli* - the world's most popular model organism
Daniel P Brink, Division of Applied Microbiology, Lund University.

### Intended Learning Outcomes:
**1. To gain experience in using the following libaries:**
- [matplotlib](https://matplotlib.org) for visualizing 2D and 3D data. 
- [NumPy](https://www.numpy.org) for manipulating and doing operations on arrays.
- [pandas](https://pandas.pydata.org) for reading in data and represent data in tables.  
- [seaborn](https://seaborn.pydata.org/index.html) for setting nice colour palettes for matplotlib plots
- [SciPy](https://www.scipy.org) or [lmfit](https://lmfit.github.io/lmfit-py/) for fitting.
- The [os](https://docs.python.org/3/library/os.html) module for navigating between directories 

      
**2. To gain experience in using Jupyter notebook relevant features such as:**
- Marking up text and equations with Markdown and LaTeX.
- Data visualization.
- Magic commands (%).
- Inserting images.
- Built in help functionality.


**3. Understanding basic scientific techniques and models such as:**
- Loading and parsing datasets containing microbial growth data.
- Processing multiple biological replicates from the same experiment. Calculation and visualization of averages and standard deviations. 
- Visualizing microbial growth data and calculation of growth rates
- Evaluating mathmatical expressions.

**4. Basic python scripting such as:**
- Datatypes and objects.
- Datastructures.
- Loops.
- Functions.

**5. Searching documentation and help online** 

**6. Generation of publication ready figures.**

### Imports

In [ ]:
from IPython.display import Markdown, IFrame, Image
import ipywidgets as widgets 

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns # the seaborn package can be used to set good-looking colour palettes for plots

import os # Later on in this notebook (Sections 2 and 3), we will moving across directories using the os module.
from os.path import isfile, join

oldwd = os.getcwd() # This line saves the path to the starting directory where the notebook is located, so that we can easily return if needed.
print(oldwd)

In [ ]:
# Run this code block to reset the working directory (wd) if something goes wrong when running code blocks containing the os.chdir().

print(os.getcwd()) # prints current wd 
os.chdir(oldwd)    # resets the wd to the starting folder in which the notebook file (.ipynb) is located
print(os.getcwd()) # prints the current wd after resetting to the starting folder. Thus, it should print the path to the starting folder

## Introduction

### The notebook

In this project, you will work with real growth data from the bacterium *Escherichia coli*. You will practice loading and handling large datasets split over multiple files, working with data from replicated experiments, and making plots and performing calculations that are commonly used for microbial growth data. The notebook is divided into 3 main sections:

1. [Plotting growth data from *E. coli*](#section1)
2. [Automating the processing of multiple datasets using loops](#section2)
3. [Calculating microbial growth rates](#section3)

This notebook has been designed in a way that will make us practice a concept known as *stepwise refinement*. This is a fancy way of saying that we can first design several simple components that we want our final script to have, and then bit by bit combine the components into a more complex project. This means that instead of asking you to building a complex script from the start, we begin by implementing one of the components we would like our final code to have. We test it and after we have made the component work like we desire it to, we can take the code and use it to implement new functionalities to our project. In this manner, we will eventually be able to reach fairly complex pieces of code, with the added benefit that we have probably understood each step of the code along the way! 

Chances are that as you are working on the notebook, you will not think too much about doing stepwise refinement. But as you reach the more challenging tasks later on in the notebook, you will hopefully be able to use the knowledge you have gained along the way to help you solve the more advanced excercises.

## 1. Plotting growth data from *E. coli*
<a id='section1'></a>

As you have probably learned in your previous courses, microbes can often grow in a range of different temperatures but tend to have an optimal temperature where it grows the fastest. The span of the temperature range and the optimal growth temperature typically differs between species, and sometimes even between strains from the same species. In textbooks, the optimal growth temperature of *E. coli* is often reported to be 37 °C, i.e., the average temperature of the human body. In this notebook you will work with a public dataset containing growth data from *Escherichia coli* strain K12 NCM3722 (Katipoglu-Yazan et al., 2023). The cells were grown in a minimal medium with glucose as substrate at 19 different temperatures ranging from 27 °C to 45 °C. Using this dataset, we will investigate how temperature affects the growth of *E. coli* and see if it really grows the fastest at 37 °C. 

Start by downloading the dataset called *Ecoli_MicroplateGrowth_Filtered.zip* from this link: https://doi.org/10.57745/GCKG7W

Copy the .zip file to the same directory that you have downloaded this notebook to. Unzip the archive to a new directory called *ecoli_growth_data*. (You are of course free to name the folder whatever name you want, but please note that in the code examples below we will assume that the 19 files from the zip are located in a directory named *ecoli_growth_data*.) Please make sure that this new directory only contains the .csv files and nothing else. This will make our lives easier when we later on in the notebook will work with automating the loading of the files.

The growth data we will be using was captured using 48-well microtiter plates (Figure 1) at 19 different temperatures ranging from 27 °C to 45 °C. Each temperature setting was assessed in 40 replicates, i.e., 40 of the 48 wells were occupied with cells. Each well typically has a maximum volume of 400µl, and to not risk samples spilling over from one well to another when the plate is agitated for the sake of mixing, each well would probably contain 2-300µl of culture. The cells were grown in a minimal medium with glucose as substrate. Sterilized growth medium was added to the wells and a pre-culture of *E. coli* cells were inoculated to a low initial biomass concentration. The plate was then incubated in an automated plate reader that controls the temperature, shaking, and measurement of optical density (OD). The cells eventually started to duplicate themselves and there was be an increase in the number of cells, i.e. growth. The growth was be measured as discrete data points, but at a fairly frequent measurement interval. In this dataset, the OD was measured every 10 minutes.

<br>
<center><img src="images/fig1.png" width="400"></center>
<p><center><b>Figure 1:</b> An overview of a 48-well microtitre plate. Source: <a href="https://commons.wikimedia.org/wiki/File:48-well-plate.svg">Wikimedia Commons</a> </center></p>

**(1) Biotechnology task:** <br>
What is Optical Density (OD)? How is it measured? What alternative methods exist to measure biomass? What are the benefits and drawbacks of OD compared to the other methods?

In [ ]:
# -- YOUR ANSWER HERE --
# ---------------------

### 1.1. Load the data using Pandas and make a first plot

In this first section of the notebook, we will work with the data captured at 37°C. This data is stored in the file named *Ecoli_MicroplateGrowth_37C_Filtered.csv*.<br>

To load the data we can use the read_csv() function that was introduced in the notebook from the Pandas lecture:

```python
df = pd.read_csv(r'ecoli_growth_data/Ecoli_MicroplateGrowth_37C_Filtered.csv')
df # display the dataframe
```

Here we tell Pandas to look in our current directory for for a subdirectory called *ecoli_growth_data/* that in turn contains a file named *Ecoli_MicroplateGrowth_37C_Filtered.csv*. The whole string, *ecoli_growth_data/Ecoli_MicroplateGrowth_37C_Filtered.csv*, is known as a path. Specifically, it is a *relative path*, since it only tells how to get to the new directory starting from the current directory. An *absolute path*, on the other hand, can be reached from any directory in the file system; for instance: *C:/Users/Daniel/Desktop/python_course/ecoli_growth_data/Ecoli_MicroplateGrowth_37C_Filtered.csv*. The examples in this notebook will be using relative paths.

*Bonus comment for those who are interested in a little more advanced discussion:*<br>
Forward (/) and backward (\\) slashes have special functions in many programming languages. Backslashes are often used to enable the printing of special characters, tabs, and newlines in a string; this is known as [string escape](https://www.w3schools.com/python/gloss_python_escape_characters.asp). The issue is that in Windows, paths are by default written using backslashes, which can lead to errors when Python interprets the backslash of the path as an escape character. Unix systems, such as Linus and Mac OS instead use forward slash for paths, so you might get errors when if your code was developped on one type of OS but run on another. The *r'* notation used in this notebook tells Python to treat the slash (in this case a forward slash) as a raw string, which means that what you typed is what the computer will interpret. Depending on the operating system you are currently using (Windows, Mac OS, Linux distrubutions) you may or may not get an error by not using the *r* when you declare the path to a directory. (Windows users could probably omit the *r* and still have a functional code). It doesn't hurt to include it, though, so let's do that for good measure.

**(2) Python task:** <br>
Load the data from the 37°C experiments as a Pandas dataframe using the code above. Display the dataframe so that you can have a look at the data.

In [ ]:
# -- YOUR CODE HERE --
# ---------------------

The dataframe loaded from *Ecoli_MicroplateGrowth_37C_Filtered.csv* consists of 125 rows and 41 columns. Each row corresponds to a time point in which the whole microplate was scanned. The column "Time_in_hr" shows the time of capture. <br> <br>
The other columns contain the 40 different biological replicates of the same strain. The numbering of the replicates is based on the well coordinates shown in Figure 1 above. This means that in each .csv file there are: <br>

$$
  40 \text{ replicates} \times 125 \text{ time points} = 5000 \text{ data points}
$$

The size of this dataset is thus a good example of case where we will benefit a lot from using a systematic programming approach to processing, plotting and calculation of key parameters. By working with this notebook you will hopefully understand the benefits of using a progamming approach to analysing your data. Imagine handling 19 files that each contain 5000 data points using Excel... It is doable, but it will likely take a lot of effort.


**(3) Python task:** <br>
The headings of the columns in a data frame often contain valuable desciptions of the data it contains. In the previous task, you most likely did not get a display of all the headings in the dataframe - it might have been truncated. Let's practice how to extract all the headings. Make a list of all the column headings from the dataframe. Compare the headings to drawing in Figure 1 above. Does it match? Are all wells represented, or are there some wells that are not included?

In [ ]:
# -- YOUR CODE HERE --
# ---------------------

Now let's try visualizing some of the data! There are 40 replicates in each .csv file, so let's start by plotting one of the replicates, e.g. A1. Plot the *Time_in_hr* column on the X-axis versus the OD values from the A1 replicate on the Y-axis. Try to reproduce this figure:

<center><img src="images/fig2.png"></center>
<p><center><b>Figure 2:</b> <i>E. coli</i> growth at 37 °C from replicate A1. </center></p>

**(4) Python task:** <br>
Plot the results from the A1 replicate (Time versus OD) using the matplotlib syntax you have learned in the course. Add axis labels and a title. Use a marker, such as 'o', to show that the data contains discrete datapoints.


In [ ]:
# -- YOUR CODE HERE --
# ---------------------

**(5) Biotechnology task:** <br>
Microbial growth curves can be divided in different phases based on the rate of growth within each phase. What are the names of the different phases of the growth curve?

Hint: if you need to refresh your memory, you can read the Wikipedia article: [Bacterial growth](https://en.wikipedia.org/wiki/Bacterial_growth). <br>

In [ ]:
# -- YOUR ANSWER HERE --
# ---------------------

### 1.2. Processing and plotting data from multiple replicates

A characteristic trait of biological systems is that they are *noisy*. This means that there can be a high variability between multiple repeated measurements of the same parameter. In microbiology, even two cells derived from the same parental cell can have small differences in their phenotypes depending on the local environment and the genotype of the cell (i.e., mutations). For instance, one cell might be at a position in the culture medium with a slighlty lower sugar concentration, and consequently grow slightly slower than its sibling cell that is positioned in a place with a little higher availability of sugars. Situations like this can result in that the signal of the parameter that we want to measure can have a high variability. When the variability becomes too high, we risk getting more noise than signal in the system we're measuring and, as a consequence, we will have a hard time interpreting the results.

Because of the inherent variability in our world, all scientific experiments - be it in natural sciences, medicine, or social sciences - requires replicates and statistics. Since biological systems can have an especially high variability, biological research requires us to perform many replicates. Performing experiments in replicates also allows us to do statistical analyses of the data to assess the variability and evaluate if a certain response is significantly different from another. In this notebook we focus on basic ways of handling data from multiple replicates using averages and standard deviations. (We will not go into statistical tests such as the *t-test* or *ANOVA*, since this is not a course in statistical analysis. However, for those of you who are interested in such tests, the scipy package comes with functions to perform different statistical tests.)

Furthermore, in the biological sciences, we make a differences between *technical replicates* and *biological replicates*. 

A *technical replicate* means that a measurement is performed multiple times using the same instrument and sample. For example, we inoculate a single colony in a shake flask, incubate it and let it grow for a while and then take a sample and measure the biomass content. We can measure this sample multiple times in the same instrument. Most likely we will get slightly different absolute numbers each time. This is the technical variability of the sample. It shows the variability of the method that was used, for instance how much a spectrophotometer differs between multiple measurements of the same sample. If we measure the sample multiple times, we can calculate the average value and the standard deviation of all the replicates. Ideally, we would like the method to produce similar results each time, i.e. we would like the standard deviation between the measurements to be as small as possible. Most mathematical software has built-in functions for taking the average and standard deviation. When working with Pandas dataframes, we can use .mean() and .std() to do this.

A *biological replicate*, is when we compare samples that are biologically separated. When working with single cells, we do this by using a different cell culture for each sample. The easiest way to achieve this is that we take a new colony from our agar plate for each culture that we inoculate. Then we prepare our sample just like we would for the technical replicate and perform the measurement. Since biological systems are known to be noisy, the biological replicates will typically have a larger variability than the technical samples. To be able to show that something is biologically reproducible, we typically first use the technical replicates to ensure that the measurement method is reliable, and then use the biological replicates when performing the statistics.

The metadata from Katipoglu-Yazan et al. (2023) dataset is a little vague on whether the replicates are technical or biological replicates. However, it says that the measurements for the different temperatures were made using "40 replicated cultures", which hopefully means that all 40 replicates were biological, i.e. each coming from a different starting colony on the agar plate. For the sake of this notebook, we will assume that the replicates in the dataset are biological replicates.

OK, let's have a look at the variability between the replicates in your dataset!

**(6) Python task:** <br>
Plot the OD results from four different replicates - A1, B2, C3, and D4 - as individual curves in a single plot. Use a different colour for each replicate. Use a marker to show that the data contains discrete measurements. Add labels to the axes, a legend for the curves, and a title to help the reader understand the plot.

In [ ]:
# -- YOUR CODE HERE --
# ---------------------

**(7) Biotechnology task:** <br>
Look at the plot you just generated. Do you see any differences between the replicates? If yes, what do you think is the reason?

In [ ]:
# -- YOUR ANSWER HERE --
# ---------------------

As mentioned above, a common approach of handling multiple replicates is to calculate the mean and standard deviation of all the replicates. In short, the thinking is that if the standard deviation from the mean is small enough, the reproducibility of the experimental setup can be considered good. Let's calculate the mean and the standard deviation for the replicates in the 37 °C file.

**(8) Python task:** <br>
Using the dataframe we created earlier as a starting point, create a new dataframe that contains the time, OD of each replicate, the mean of all replicates and the standard deviation of all replicates <br>

Hint: Remember to not include the first column (Time_in_hr) in the calculations of the mean and the standard deviation! That column contains the time of each sample, and not the optical density...

In [ ]:
# -- YOUR CODE HERE --
# ---------------------

Now we have a dataframe containing the mean and standard deviation from all the 40 replicates at 37 °C. Let's investigate how the mean of our data looks like.

**(9) Python task:** <br>
In a single figure, plot the mean of the replicates together with the data from replicates A1, B2, C2, and D4. Plot the mean in black colour so that is stands out from the other curves. Add labels to the axes, a legend for the curves, and a title to plot to help the reader understand the plot. In the label for the mean data, please specify that the number of replicates used to calculate the mean (n=40)

In [ ]:
# -- YOUR CODE HERE --
# ---------------------

**(10) Biotechnology task:** <br>
In the figure you just plotted, the mean is probably closer to the A1, B2, and C2 replicates than to the D4 replicate. What does that tell us about the overall performance of the bacterial cells from this experiment (37°C)? Do you think that the mean is a good representation of the replicates in this dataset?

In [ ]:
# -- YOUR ANSWER HERE --
# ---------------------

An issue with using the mean of all our replicates is that it does not show how much the different replicates varied, only how they performed on average. To better be able to show that the data contains variability, it is common to add the standard deviation as error bars to our mean data. In the figure below is an example of how these plots often look like. 


<center><img src="images/fig3.png" width="500"></center>
<p><center><b>Figure 3:</b> Example of a plot of the mean values of discrete datapoints and error bars with the corresponding standard deviation.</center></p>


One way to plot error bars with matplotlib is to use this function:
```Python
ax.errorbar()
```

You can read the documentation <a href="https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.errorbar.html#matplotlib.axes.Axes.errorbar">here</a>.

**(11) Python task:** <br>
Plot the mean of the replicates and display the standard deviation as error bars.

Hint: the arguments *linewidth* and *capsize* of ax.errorbar() can be used to control the look of the errorbars

In [ ]:
# -- YOUR CODE HERE --
# ---------------------

**(12) Biotechnology task:** <br>
Your plot probably showed that the standard deviation was smaller at certain time points and higher at other time points. Can you come up with any biological explanation as to why this could be?

In [ ]:
# -- YOUR ANSWER HERE --
# ---------------------

And just like that, we have created a small piece of code that reads the 40 replicates x 125 time points = 5000 datapoints from the 37 °C dataset, calculates the mean and the standard deviation of all the replicates for each time point, and plots the results. Very handy!

We should keep in mind that the code we have produced will only work for .csv files structured exactly like that data from Katipoglu-Yazan et al. (2023), i.e., each column is a biological replicate and each row is a new time point. Please keep in mind that if you get OD data from another experiment made by a different person, the data might be sorted in a different way... This highlights the importance of knowing how your indata looks like, and that you may or may not need a sorting step before you send the data to your script. There is no standard format for OD data! But sorting it in the way Katipoglu-Yazan et al. (2023) had done comes very naturally for working with Pandas dataframes. Also keep in mind that if you change to another programming language, there might be other preferred ways of sorting the data - if this had been a course in the R language (another popular language in the biological sciences), we might have wanted to sort and label the data in a slightly different way.

**(13 BONUS) Python task:** <br>
From our previous plots we have observed that some replicates were more outliers than others. We have yet to plot all replicates, so let's do that in this bonus task. In a real-life project, plotting all replicates would probably be the first thing we would do so that we could get a feeling for our data, but since the focus of this notebook is to practice Python, we skipped that step earlier.

The challenge of plotting 40 replicates in a single plot is that the figure will be hard to read: there will be many colours and markers. Instead, it might be more useful to split the data into subplots. For instance, you could make one subplot per row in the microtiter plate, i.e., one plot for A1-A6, one for B1-B6, etc. Remember that rows E and F in the dataset we are currently working with include eight datapoints and not six.

Is there a pattern in the data? Are the replicates from the same row of the microtiter plate reproducible? Are the replicates from the same column of the microtiter plate reproducible? If you want, you can load the data from another replicate too, and check if any patterns you observed in the 37 °C data are also present there.

Compare the variability of the replicates you have plotted to the standard deviations from the plot with the mean OD and standard deviation you just made in Task 11. What are your thoughts?

In [ ]:
# -- YOUR CODE HERE --
# ---------------------

## 2. Automating the processing of multiple datasets using loops
<a id='section2'></a>

By now we have written code for loading the 37 °C growth data from its .csv file, calculating means and standard deviations, and plotting the data. Since microbial growth data is a very common data type in microbiology, this type of code is useful to have. You will probably be able to reuse this code in future courses and project, if you want. In this section of the notebook, we will work on making the code even more useful. 

As you probably remember from when you unzipped the growth data, the zip archive contained 19 files. If we want to make the same plot that we made for the 37 °C data (Task 11), we could use the code we have already written and repeat it manually for each of the 19 files. There is nothing wrong with processing data in that way. But it is time consuming, cumbersome, and prone to error. Instead of doing this manually, we could write a loop to have Python do the work for us. We would like the loop to find all the .csv files in the directory we made, then load the .csv files one by one, perform the calculations and plots, and then repear this process for the remaining .csv files.

To do this, we will make use of a Python module called *os* that allows us to e.g. move between directories on the hard drive programmatically. In particular, we will use two functions to do this: 

```python
os.chdir()

os.listdir() 

```

Using the os module is a powerful way of interacting with files on your hard drive. But it can also be tricky to keep track in your head of which directory you are in at a given line of code. If you are at anytime lost, use the function *os.getcwd()* to display the folder you are currently in, and use the function *os.chdir(oldwd)* to go back to your starting directory; the variable oldwd contains the path to our starting directory and was declared in the import section at the top of the notebook. If you get an error in your code block after changing directory, the previous code blocks you worked on in the notebook might no longer work since you are now in a different directory. If that happens, don't panic, and just run the command *os.chdir(oldwd)* and you should be able to go back to the starting directory. You can also add the code *os.chdir(oldwd)* as the first line in every code block in this notebook, if you want. This will make sure that you always start in the starting directory each time you execute a code block.


**(14) Python task:** <br>
Test the following code for yourself. By using *os.listdir()* we can get a list of all the files in a given directory. In the examples below we will use it on the *ecoli_growth_data* subdirectory located in our starting directory. Note that for this this to work smoothly, the directory should not contain any other files than the ones that we want to process.

What result did you get? Did it contain the file names of all 19 .csv files you downloaded earlier?

In [ ]:
# -- YOUR CODE HERE --
# ---------------------
os.chdir(oldwd)                           # first of all, tell Python to go to the directory where the notebook is located. 
                                          # We defined this variable in the Imports section in the beginning of the notebook.

my_files=os.listdir(r'ecoli_growth_data') # Look in the folder we made called ecoli_growth_data and make a list of all the files therein. 
                                          # The path to this folder is [oldwd]/ecoli_growth_data
                                          # Please make sure that this folder only contains the .csv files from the zip file we downloaded and nothing else! 
                                          # Doing that will make our lives easier :)
            
print(my_files)

**(15) Python task:** <br>
Now that we know how to get a list of all the files in the directory, let's try looping over them. Here is an example code that loops over each file name in the list *my_files* and loads the corresponding .csv file.

Add your own code in the loop below so that the name of the file and the first few rows of the data frame is printed in each iteration of the loop.

Hint: *df.head()* prints the first rows of a Pandas dataframe

In [ ]:
# -- YOUR CODE HERE --
# ---------------------

os.chdir(oldwd)                           # first of all, tell Python to go to the directory where the notebook is located. 
                                          # We defined this variable in the Imports section in the beginning of the notebook.

my_files=os.listdir(r'ecoli_growth_data') # Look in the folder we made called ecoli_growth_data and make a list of all the files therein. 
                                          # The path to this folder is [oldwd]/ecoli_growth_data
                                          # Please make sure that this folder only contains the .csv files from the zip file we downloaded and nothing else! 
                                          # Doing that will make our lives easier :)

os.chdir(r'ecoli_growth_data/')           # Then, before we run any loops, we need to change the directory to ecoli_growth_data. 

for file in my_files:
    df = pd.read_csv(file)
    #Your code here
    
    
    
    
# at the end of your code block, add this for good measure:    
os.chdir(oldwd)           # This makes sure that you go back to the starting folder after the code has finished running.

OK, now that we have gotten a hang of the basics of looping over our .csv files, let's add some code to apply some of the data processing we worked with in Section 1.

**(16) Python task:** <br>
Write a loop that goes through each file in the directory in which you put the *E. coli* growth data files. Then, for each temperature (=.csv file) calculate the mean and standard deviation of all the replicates, and plot the mean of each replicate with the standard deviation as error bars. Plot all the results in a single figure.

Hint: For the contents of the loop, you can most likely reuse part of the code you wrote in Section 1.

In [ ]:
# -- YOUR CODE HERE --
# ---------------------

You might notice that the colours of the curves can become a little hard to distinguish from each other when you plot 19 curves in the same plot... One way of handling this is to split the results into two or more subplots. Another way is to try to find a more suitable colour palette. Here is an example that uses the seaborn package to set the colour palette.

```python
import seaborn as sns

fig=plt.figure(figsize=(30,10)) #create a frame, as usual
ax=fig.add_subplot()

colors = sns.color_palette('hls', 19)   #call seaborn to create a palette of 19 colours.
                                        #(We have 19 .csv files, hence the number 19). 
                                        #You can change 'hls' to another seaborn palette if you want
        
ax.set_prop_cycle('color', colors) #send the colour palette to the axes 

ax.errorbar() # plot the data as you normally would
```

You can read more about the built-in palettes of seaborn <a href="https://seaborn.pydata.org/tutorial/color_palettes.html">here</a>. Some examples include *'hls'*, *'husl'*, *'Set2'*, and *'magma'*. Play around and see if you find any palette you like!

**(17) Biotechnology task:** <br>
Microbes have a range of temperatures in which they can grow. If the temperature of the environment becomes too low or to high for the microbe, the cells will become stressed and eventually cease their propagation. Looking at the graph you have just generated, at which temperatures does the growth of this *E. coli* strain seem to be affected? Does it look like this *E. coli* strain grows the fastest at 37 °C?

In [ ]:
# -- YOUR ANSWER --
# ---------------------

## 3. Calculating microbial growth rates
<a id='section3'></a>

The plots we have made so far have probably given you an indication of how this particular *E. coli* strain grows at different temperatures. Remember that the only difference between the data from the .csv files is the temperature - all other parameters were kept constant in all experiments. The optimal temperature for cultivation of *E. coli* is stated by many textbooks to be 37°C, which happens to be the average temperature of the human body. In this section, we will use the fact that our dataset contains growth data from 27-45°C to see if the maximum growth rate of this strain actually occurs at 37°C.

The visual inspection of the plots indeed gives us a lot of information. In addition to plotting the data, it is also common to quantify the growth by calculating the growth rate, $µ$. Specifically, we often want to calculate the maximum growth rate, $µ_{max}$, from different growth curves and then compare the values. Here we will eventually calculate and compare the $µ_{max}$ from all the 19 temperatures from our dataset.

### 3.1. The maths behind an exponential growth curve 
As you discussed yourselves in Task 5 back in Section 1, microbial growth curves consist of several different growth phases. A common way to describe the growth during the lag phase and the exponential phase is by using exponential models of the form:

$$
  Y=Y_{0} e^{µ (t-t_{0})} 
$$

Where $t$ is the time in hours, $Y$ is the number of cells at time point $t$, $µ$ is the growth rate at that time point, a $Y_{0}$ is the number of cells at the starting time point $t_{0}$.

This means that we can solve for the growth rate ($µ$) by:
$$
  µ=\frac{ ln( \frac{Y}{Y_{0}} ) }{t-t_{0}}
$$

According the properties of logarithms, we can re-write the equation as:

$$
  µ=\frac{ ln(Y) - ln(Y_{0}) }{t-t_{0}}
$$

This is the equation we will be working with in this section. You can compare this to the general formula for finding the derivative of a point on a curve, and hopefully see that they are very similar: 

$$
  \frac{dY}{dX}=\frac{Y-Y_{0}}{X-X_{0}}
$$

That means that to find $µ$ at a given point in a microbial growth curve, we need to be able to calculate the slope of the natural logarithm of the biomass curve at that point.

One way of finding $µ$ from a microbial growth curve is plot the natural logarithm ($ln$) of the cells versus time. We call this a semi-log y plot. 

We are often interested in finding the maximum growth rate µmax, which is the point of the semi-log y plot with the largest $µ$ (steepest slope).


**NB! in many programming languages, the function for the natural logarithm is *log()* and not *ln()***. The base-10 logarithm is often written as *log10()* in most programming languages. Likewise, the base-2 logartihm is written as *log2()*. The benefit of working with the natural logarithm is that it can be easily used to transform exponential functions, since we can use the definition that states that:

$$
  ln(e) = 1
$$


**(18) Python task:** <br>
The first step in using the ln-method for calculating the maximum growth rate ($µ_{max}$) is to tranform the OD values to ln(OD). Calculate the natural logarithm of the mean OD from the 37 °C dataframe, and plot the results. 

Logarithm transformed plots can sometimes look very similar to their non-logarithm counterparts. Once you have made the plot, note the scale on the y-axis. Compare it to the non-logarithm version of the figure you made earlier - what is the difference? 

In [ ]:
# -- YOUR CODE HERE --
# ---------------------

**(19) Python task:** <br>
Using the ln data of the mean OD from the 37°C data, calculate the $µ_{max}$. Use the equation above that states that $ µ=(ln(Y) - ln(Y_{0})) / (t-t_{0})$.

Hint: one way of approaching this is to first calculate the slope ($µ$) for every point on the curve. Once all the µ are known, we can e.g. use the Pandas function .max() to find the maximum µ value.


In [ ]:
# -- YOUR CODE HERE --
# ---------------------
# y=y0*e^(µ*(t-t0)) 
# => µ=(ln(y)-ln(y0))/(t-t0)

**(20) Biotechnology task:** <br>
The formula we have been using to calculate exponential microbial growth can also be used to calculate how long it takes for the total amount of cells to double. This is known as doubling time, or generation time. How is the doubling time calculated? Use the $µ_{max}$ value you calculated in the last task to calculate the corresponding doubling time.


In [ ]:
# -- YOUR ANSWER --
# ---------------------

In the previous two tasks, we used the mean OD value to calculate the $µ_{max}$. What could be the issue of calculating the $µ_{max}$ of the mean OD? Do you think that we would get another $µ_{max}$ estimate if we first calculate the $µ_{max}$ of each replicate, and then take the mean of the $µ_{max}$ values from all the replicates?

**(21) Python task:** <br>
Calculate the $µ_{max}$ for *each of the replicates* in the 37 °C data, and the time at which it occured. Then calculate the mean and standard deviation of the $µ_{max}$ values from all the replicates.

Did the $µ_{max}$ occur at the same time for all replicates? How did the mean of the $µ_{max}$ values calculated from each of the replicates compare to the $µ_{max}$ calculated from the mean OD?

In [ ]:
# -- YOUR CODE HERE --
# ---------------------


### 3.2. So what is the optimal growth temperature for *E. coli* according to this dataset?

**(22) Python task:** <br>
Write a code that reads all file names in the directory in which you have stored the 19 *E. coli* growth data files. Then write a loop that reads each of the growth data files, calculates the $µ_{max}$ of each *replicate* each file, calculates the mean and standard deviation of the all the $µ_{max}$ values in the file, and finally stores the calculated values (for instance as rows in a data frame). After the loop has finished the calculations, plot the temperature versus the $µ_{max}$ and standard deviation.

Hint: If you want, you can take the code from the previous task and use it to construct a function that you then can loop over.

In [ ]:
# Use the code from previously to loop over all .csv files
os.chdir(oldwd)
onlyfiles = [f for f in os.listdir(r'ecoli_growth_data') if isfile(join(r'ecoli_growth_data', f))] # code to get only the file names and not the subdirectories from a dir

# -- YOUR CODE HERE --
# ---------------------


**(23) Biotechnology task:** <br>
Now that you have plotted the average $µ_{max}$ and standard deviation for each temperature in the dataset, what did you find? Did the fastest maximal growth rate occur at 37°C like the textbooks often state? 
<br> - If yes, explain why this temperature would be beneficial for the growth of this bacterium. 
<br> - If no, what could be the reason why? 
<br> - Where there any results in this plot that were different from what you had expected?
<br> - Compare this figure to the plot you made with the mean and standard deviation in Task 16 in Section 2. Do the results match?


In [ ]:
# -- YOUR ANSWER --
# ---------------------

### 3.3. (BONUS) Fitting curves to the experimental data

As we have discussed in this chapter, we often describe microbial growth as an exponential function, but that is only true until the end of the exponential phase. In fact, what we often see is more of an S-shaped curve, i.e., a <a href="https://en.wikipedia.org/wiki/Logistic_function">logistic curve</a>. In our data, we also have a decay phase after a short stationary phase, which may or may not be an issue for fitting a logistic model. Most likely, we will have some troubles if we try to fit a model to the whole growth curve, regarless of the model we choose. But if we only focus on the data up until until we reach the stationary phase, we can use an exponential function well enough. If we try to fit this exponential function: 

$$
  f(x)=a e^{b x} +c
$$


to the data up until the end of the exponential phase, we can get a figure that looks something like this:

<center><img src="images/fig4.png"></center>
<p><center><b>Figure 4:</b> Example of an exponential model fitted to the lag and exponential phase of the data from the 37 °C experiment. </center></p>


In this data, the end of the exponential phase seems to occur around row 34-37 in the 37 °C data. A piece of code to drop all rows in the dataframe after row 36 and save the modifed data as a new dataframe could look like:

```python
df_copy=df.drop(df.index[36:])
```


**(24 BONUS) Python task:** <br>
Plot the mean OD data from the 37 °C dataset as discrete points in a scatter plot. Then, using the same data points that you used to plot the data, fit a curve using the exponential model $f(x)=a e^{b x} +c$. Once you have the fitted curve, the parameters of the fit to get an estimate of the $µ$. Did the $µ$ you calculated using the curve fit differ from the $µ_{max}$ you calculated using the ln-method? If yes, why do you think that there is a difference?

Hint: 
If you plan on using *lmfit* or Scipy's *curve_fit*, please remember that it requires you to provide it with initial guesses for the parameters of the mathematical function you want to fit.





In [ ]:
# -- YOUR CODE HERE --
# ---------------------

## Graphical excellence

**(25) Python Task:** <br> 
To finish up the work we have been doing in the notebook, we will sum up some of the main results from our analysis into a single publication-ready figure. This means that the figure is ready to be used in e.g. in a scientific report, power point presentation, or conference poster. Reproduce the plot shown below (seaborn was used to set the colour palette). You may also make improvements to the figure if you want. Perhaps you find that there are too many curves to plot in a single figure with these number of colours - you may want to split the data into two subplots, or maybe even skip some of the curves. Anyway, the resulting figure should be something that you would gladly hand over to your supervisor! <br>

Do you understand the plot? It should be apparent if you completed all the previous exercises. If you are not sure, please ask your instructor!.

<center><img src="images/fig5.png"></center>
<p><center><b>Figure 5:</b> Main results from this notebook. </center></p>


In [ ]:
# -- YOUR CODE HERE --
# ---------------------

# References

Katipoglu-Yazan et al (2023). Data on the influence of temperature on the growth of *Escherichia coli* in a minimal medium containing glucose as the sole carbon source for the joint computation of growth yields and rates at each temperature from 27 to 45°C. https://doi.org/10.57745/GCKG7W

# Suggested answers to the tasks

### How to get the answers:
Enter the password in the next code block. If the password is correct, you can execute each of the code blocks below to get decrypted suggestions for how to solve the tasks.

```python
fernet = Fernet(b'!!!TYPE PASSWORD HERE!!!')
```
Note that the password needs to include a b' at the beginning and end with a ' .

In [ ]:
from cryptography.fernet import Fernet
fernet = Fernet() # enter password here!

**(2) Python task:**

In [ ]:
task2_encrypted=b'gAAAAABkmargg77A-IUDrcUvYmmPNBc9-laeESHOAp8GdrE8b_lf9gzo4hB1VOfaApkc4ThxG1WeP13h2CqbPPGzhtQhqvYybkwx5uByxmpPyDOI1T5zs4lmzHXCXPoUe8Mk7IBqLyF-wnK8JsOmrb3SKA6czHK6rvGucsH8AKhyf7cphnj_kwxTb2PYNGSgiYJum9v2Ug0zda9ZTdE47GSGFGg5Ao6gAq5m78gGYThVkY_Ibw6fJA5Ezw1TZIR_-z5RfP-XuVTCGIfp1mACb_3_mIjqq2T1Eg=='
task2_decrypted = fernet.decrypt(task2_encrypted).decode()
 
print(task2_decrypted)

**(3) Python task:**

In [ ]:
task3_encrypted=b'gAAAAABkmargLwUkfjyvskh1U3j24f2rHqkS0rcsG2giiK3a4TRl1l6caLKrnH-anc1MGmX-uZ1ChiorusK4Bemi-7k2fyDC1DReQnuZ7I8T_NyAJLF__7tX5o6kwYpKmfL7X6mkoXG_aG2GVxypbzMLo9-7fzmNow90MMsOWVN-lyqlID6tvyy7pt0sjV233AvU0jqpPOHgnoZbtiPwFnD-NgnDvTKf8UrUrbiURyrQJAVS2rAQIZNxbB7Bw5dk32oBsj5qzzLo3GNfvnAmGki1EqCrUXFhXQmGyx0w7dzG8Zav0q3pJKA4EueIuogx-xWK4TyFp3MW'
task3_decrypted = fernet.decrypt(task3_encrypted).decode()

print(task3_decrypted)

**(4) Python task:**

In [ ]:
task4_encrypted=b'gAAAAABkma5VFCL6S8uzcFevSysi23iEj5smuh4yXrvAC4lmE8c90NSPYf6lZF51tvKNir2zXFb0q4NMAdnnda29RwBsAtGz10W1ZTWYvljI27PYb8EG09yaojlOUPYU4ZIVokfCHyliSrG7sARgQHVHuu4Khv3hrTKXFuskNyx2NjobL83x3AmBfwZZDoEKSdpbqUvtWsTPnxugJBpM_DQQKsXCfCSJV7_ICyjrB312ABMgmRDfP86LYqFnmoSgj7Y_umfqCKANnXDD6A1mD6wtL5YNIlXGXptCV1Ot1pF5nhCXvSpGaGJspDJd89CJ40mKbXuX0gOAfeYS2riO847YDlNk0k1ouKDqSlDh8DgUB2OUNZT9jIzdCb7HC8vmwccLIPjqESnSSaEabKUf1c2xJ6RTrQkFNt6As1b-wzBPW_MVN1nuY-3Qo1LzHVFGXJ5Xl2Lnc0onrAv4BdpdaLnkrKd5CCQwc6ajXRkBzThVQ--as4mM02My5s67JCJ8t7HJrB_aZEiC75aBzqW1ra9cMvb1yPRPOOs9zzgXiw8QZZCy_oWcNZtH3vGUDey9eWCOISVpoLPa'
task4_decrypted = fernet.decrypt(task4_encrypted).decode()

print(task4_decrypted)

**(6) Python task:**

In [ ]:
task6_encrypted=b'gAAAAABkma3k1_IqwolEQ2bEabZD6B77rVgKDzTWBLpEwqR6RqCtavyD2V5K9RbRMj_OyMx72cru6Wiqrley1N6W-PKWUi9ZX5UzkZo4n-afD217PbNY-fNBEOeCRs1oeNrP4DBCpst3VRpwzgeoIj8VpCIVkpPrh4tYzy8cmrSbT9NvCvtXcsYGVuOKoc6_B9WI0j0b3jSZwb1g11qOYWCtcqkU-vWMrzoN1hDrX4fH15AzgHfhQgdxOBUrfT2gZigfDIVQ92RvTlFa0vJmX-HOmwrgU_I-Q0KThda9U_tQ86PwlrCJD4D9T7LWU7di2hALbeOYAhouv-a9OCQgD8MXtQDfVBnR8VX0i598AbI3teBSCdSr1bXN8jw7TfT7lGONL-9sSMURMlzuX85yJIhG6aMIPs4KSfk6zVvyDBTR3o4y1DVWTEDlDW0Y0EfOv129N99bHAOs2ez2gL5cGNpD94cSsnA7U32Z7P9exDHsLkQLLR63dMZxIOR_eIcnxRzKmqlgif4d2sTO776DVK7byQVf7Rq8yRUnMGV2-beuv5hAry45E95u1YbtFmsisz5sty_KZE90wt2ZtUYje6PW1wrPnB_5dftbLkypeukoxd9LHV1A6HfHsVr1oLOtoEOK4dQSeXLwdpAIpfF33zwYgvFgxLnZJx2PyE2-CZoSufcFYy96YRQA-0FDOYgDUMORY8UW6boPAiJSEjSF-FXGPYnnqd871VRBiPCVLwPNSW-zMJ2EPrrxTz6xsOaI6KZ96o5mEhFPy3l4I6l5Q5DPhUbaDHOR_g=='
task6_decrypted = fernet.decrypt(task6_encrypted).decode()

print(task6_decrypted)

**(8) Python task:**

In [ ]:
task8_encrypted=b'gAAAAABkmawiCIsC_Rw7JvgVg8yFRt2Ozmw_FEUAzjwhhlQi8oEB_oNgOU1OQkBKDwZwZ2EuayNHhS8FSG6tVgLS8aUffQihkmTvntwtmN8VF-JpkYJSSgmLTza6_64Om0ojv6wIHeaRUkrh-MDaMOd3i3sjwqk_RK0tasl906wKukqdKYKnrRtNaXhgb6hiWoVDRkmnLVzA11wFiPDPpNf7n8YVGHBybPVL0ReFcWLOPXhDzKHHH_GQ27rlC0t5c9Ls2TAZA15f5xQrQ7BX8owyuk7QLzA9KwNyyII7TXMYfn0MrsaqmSIj3WN5XWNPcUFhNfOrCdhPDWpXAFYoXB3P_fzP5SrWF28eWydWLIlbvmVjTTo6ENUNoHMD_3BzFg8eReKARbQzckSnDYihYlMjVb0kH7lVvyJJLB12D96JS0riNGhpJf29vYV5hLmUg9M8BHEpvws_XjvyTUx5XS96jx20r9RwvX1M01NhAK-iBwJwIr5mLW-t_21_Uf8tfLloX6a7u6-elDMEc2a1bujlgj59mVREkBKjCRlHMw0vag10yoo3PGRep6yNZmk3YT9cl5XmxZS5KSEDO5RjK97_rfXn9vdXFISWNC0j5o8V2A-DbbQ9slxgjDA5aXXGb7J6aJuLyf6YAweQpFoWWTpG2h8PR36G6KEkuzcyhL7e-E-AjMKWaBcN3BHJi40s_ecXQMqqwKDdca5xxUGjzUkoUdmXRbv6csbrnP22BFUy1zIdtOjGAiIC5BeyNi-5_5BOG-6x7ed8TojUmp6K1t45F0Uly63IWSXK9gvLIHQB_7hQ0o6twRxDWXUWEhbgMi6R-nU_yckIZ7DTs1ztZFrTyjThotxl0A-4olgx7k3_x9gd_yigk2OAXaj1_Y0CM5JXAiAMa8vcQL7QrAo-EuRpTC5g4v6L2L4GE62skvaimZV07FB5z6qwQUMkxJW20XCr6yOUTQR_'
task8_decrypted = fernet.decrypt(task8_encrypted).decode()

print(task8_decrypted)

**(9) Python task:**

In [ ]:
task9_encrypted=b'gAAAAABkma6-VCr3LzYyFRh0uiuo2PjCE-QC7AjUz3EPTc8B-tfvtXSlpZl4OSdSJolLBALAax-3Auh_abRqJWZ4i761rgEMzV2fg2LTu8W_bX_I3U1Z6t9L8nTv-t8x5W11WvRlhx0_QQB5q_6VZd13hSyRlXnPFDfKF5X270gJo-x7RVZ33OK9pLl3UNwMexXROGQiCpKHO5VvZVqOAfKzaNKyLKtAo2_bqONxupqRcrOEkKGchfB--GLbxHSU_pbAsXhJ7Y4I2rs4t708Qs1QHVmqMfaTufcUx9JnKQ2CH37HsEPvnio2aoy8QvaC1BxDVWIopZH4zxdlp60LfqkzPBETxjBjahnOmxT_M1U_laRHwzEgYGsLPmgHKBD0Y5t37UjcLIDXSkucZkLQMATJgzMfRdEhiocbsrImxoiBC9qRCmny1uZWungSlZVZ-yjyNOLGicMmw7WJ_V5DBrhn55WZrLtI4hYLQgtvattSIOZT9-Gz9yrvCRfZZltrVfZDS1z45DpeUc5fFKosQ_zg1S7oCtCyzy0JUPNCd1i9A37-2zTTlgK8-LvWU3GD_06biak6Xb87pprZZiiUKSJJnQcMtLEl8n_24bTs-FKEMF1WwCjeKGP45KrTuznlsrHhdp7EETda7KVwAkUWy9ispY-g_OaRHOwtG7GEJRPfykO5RI61cuwXbEFnDrEEmaITrVbeRfrwaxa4hkADtZaP8E5llBqdSb-p5D-O0B85hXCH0L8tVJ31FkRQL_TZWYaRtZh_KbL_JmB0BmlCstNQziB4l6uMh8Cey4G3iI5kLKWJxQ-Ub3gz_aXmNeTwx9VCZhMI98lozQpdCpYwhgNn7motE28dnYDZ0OLrSbCNjvzzgR9K40kOqsytJoM-0GwlZ87C6DROZyd78H5pxsXfzrCsXQGV6nEgsm7AEbRvPWAKuOkkQ8wFlCKN8FvHrCvZ1dzFnm99B7kpx4sLtzIWaylP2uY1os9AN0-s1DtfEdSz6jQwMR-zO1XMQC0934M_PaSfaPH49ua_SdkFdHk5xojh9E2HXA=='
task9_decrypted = fernet.decrypt(task9_encrypted).decode()

print(task9_decrypted)

**(11) Python task:**

In [ ]:
task11_encrypted=b'gAAAAABkma9f5krh8gIvK_XeCzL4xlKXmr2_pxT1e4lHdtNjfd5V7JQBV5xgRYd4dAO2p_TyTNsD0XMXAN39yEZvo56RVScyHOdXKNbu8_jrtKov46cg6ZauqpQXob6K9HiDzuvh8UKQ61fHrbxJeSZ-NK1kg5wLGKDe-BC8N0TRS5hFQUvp9A_pfF0Q1JNgI0EYLMMI7qsV0rGGcJP-Alvje6kWV2HayyBWoy7mXyK0V1MZblh7aWXL2GR6sgmvWrOe1OjhNrAJdCewvBrYAvnuGQ7TlJUevmNnclFDDG4EVa9bgYHEtZxdXsPobNRW1GJ0sZcF1mq5IRk4VEMZHmmkN7CvSgelnTdfBcxpB5qtWpdr3lSQKdNzVY3VHSHY5QlP8EYneVHNxWLObs97u6tXibweTxW3IXr6w08lCxy-6wh9qP9_vd9ZNp9zxgv8Tz2dpK3SPCdfwNXpVR69wxjoCM5axWXX0L50rCLzTfyOY9DxlZmzE4BXEzw-5y5shWu-dFjdbW0FwTCYvePPdjdlcJv6TdAc1JCb_1nseIzD72jZ9rMpeFZwBPn52Yp8uI912Z9WLsOC'
task11_decrypted = fernet.decrypt(task11_encrypted).decode()

print(task11_decrypted)

**(13 BONUS) Python task:**

In [ ]:
task13_encrypted=b'gAAAAABkma-wavd7Y7Q2ZMeqwDLwJJVbQqbvpXz89c_U-iqKBOHgKMdeaGeJJI25RJXt-EnzQrBauB1FKHfhCP2BFD7Hs9SZ85pzTncjo8lhZ6gc6ZLuOTM9gkHpbfe5SLcF_H8x-9fFzbpqFS01FO36d7xbAVmyJ42tkDoJYpbDVH1tiuqjTWRWdR1ppnzcTIRoKRcaASGGHYogWVI_ysElKljj8MMZ2WyQfAB5XovNW8VMKLsOlP5nUUrRNwGBuXpfUFjBhf_s3tRt4KlI9j8O0f3Oy6rb62ej0v1zU8CrEfLpjavh7Zvafr4KLLNr7gbQVYUOT6tR1gSx5cGVwxr8z57BnAiyRXn4F3pqSBtNqxADvFmSADecPeS2SyGgPfBYlMh9L0zhWkep24CrEfPL7RLF2R7iHf8jdlMIgCxQiIqiFQFndjW0FxCLVtHGDRRx0vwxYC1ajhkW71Nz87elvOjBK-nUkp3fmCsv2MsC9oNPvNguxzMAmv4xGFOyhvwODgyE7dCA43y48i43hes7EEgqHolB1dRGO8iV77sLOZSIzHFXAfsq8MooQNAMB26oMjldaODCgKjAC1bwAyossohpKtlT7bw0pjGVAPFqwSp2TrxcD0qLJFTdiIwm1qUbRdyCHwilVPtpt17mhr-7VHQEBw6ImbAe7FO1nN8L0kmVzfDaT_fRYXSWNxNF2V3tMs3FpfojTGmUF0ImVYgf3sOXL_UzoXYvBFEm9z3ud6bDhehYfPY5E6MZWiz7D-wn6tjzrx552lb7Qu3VHbkyFCLVBJjF3-VyAx68_1-Fqgu6bDrKwYV3QxUcyfSxagMNjTCxPNfjXgd01qvAPbwJ8PDJwjdPBHdhsCa5czazr3j1M0Qfn9WjFnsB8iCpFwgBPmVTcCu_l4lqum6mq1jwNkReP--nRh6uacz2TMWqA5CZMfiWZ8a6Sr__BvbHtNCf2Q3CYSd49jS6cZvetUgwTLXsWU4OKzwT4MLc4e0zqXduuP8KBETbGRzdfkrMjW86HddFjt_rnYXzXRFCS1okLKpU4Er14kf0EGJo7XnxOvJ1QWzUw7gkvLbi0e1_dpqrkyuJJk8mPYbEQ9ockRApn1nWyXSCC7UrJUTpxAGUhH3iVsyQxsbWlCUXJiqaajwQKbhIESSE8dLlUs0Shz6DRNIvVwcIiru9Oa7QB9AQm2IRQgat9xf7FOY0JDe66rUdwasOxGLupclVrask7f9jiW7lG0Sp_zMthpNDSPee2Q7cffiUSUvEE1zPvboCEgNF5Sf_IUqj6fSsvlU0BKGPhjA8Z3pwKq3w5GRNh1ZOGKwVLuJWDjDBDfce9qwMWTnINbl2DhPj17fd9Q3FiAdqyooPywUTnrvAn3Cx7GllehPjtDWI_oG6MBNfeDmNtASD_SrD1xzNUzezPdP7JdNGNIzo2GVSI43-Uea9AFmf8Lf51IU84zQH4rRxSfwdHf2FHLQ_wY4aUp7Ludj-Ce-i7Ee3O9hSnmhINBgVobzJewMvoHZWy23D_9nqza1liZ5Wpd_q4DovAnpvFqYDRWTF39ayai782v91ziOzYq8Tr3Eg7Rf95HiGBG35ra-UTNKgSB-x52NTrgyLgSVbZO9BmtFM0kEbwP5f9o8pVgqy_oBsgVfUFBnA-O-Z8r1eUOB-GIGXp1VJJadgN0n1qw1I-IePDtJ3hKuzEWzt830wTgJFI2TAKzcYx3gVoW6AV4sGgCHy3v2I_JfAZOYTBaIHbE78bicizkds-cwcanu9f90wshZn5SDtnCvJ0-3Jjji4spdllNVJqvj926Zu8_nMKhbd5BVTNhugtB3drhkEDHO65jnCWlXTiaBgx2NuQ6lI6UCQ9QTD9ZC1BhEpg5Bf5zE9hco0dyOAJ_97DIMpK8wnC6yPlXey_MG1MpVIT_231CTH_3ye3DpYkwafLkTdUwaP4JSC_6_1NGSGzaEzdgi2g8xKl0XyciyVbTC_IaxET2YyonnmolB3T_yjCT0GHaLpaZvAwJHojueX3nOZNIIT4LNk5sRb0Df-uwYjVOoXDyA3oapCvYs-XqmaaMS_byDYHUrP2FV24hmnBqysmXkmznUY5mexZpQNl186OJkPqHNaDTIomBnQq_TyBESOuei1wNm30mSLKqG48ZkknactCBAuqpLLi40YKqyDQ-rKi9ow0R1GuOe2Ppqgl9bvwzHwrcJwI7N2qgoBf14TzS0w7ZcCTcRpNupwU5oVHRMk2QUx3pcEV9JETghvqLTddwfrY56uRasgzrUsHqhBsw5CVw8uHaOSgHyjGYO-3z1I1D2JasasPJwL2Hhaz7Jxo9UARFAu1QYk8d4j7oV2aQitQo-iBWsI--l_GhvxM6Si2tWgCEEOZPpifpVsZVsG7qkaZl9ZJjchzzJr0VZ9OMuLKC3Pt6i2L6jDordZaxl1PvVPtKh42P6PRC3Zyb3SDlnT8F3WjzbANrn6XqGEOwJAafY27I_gApjcU5pNsrwUhCh5iGpKAI1o3XibLAgnhI9ikyNQQRORICBGCPd9mgkNNrXexNuKN6U6a921s1wrnOBrusodURoC7Nywgt5-MSFJn7WUrGON7cbfKnxaoGcDlLwpBConPm5ssgR7332fs6FKReD9QSZsIYNEf3D9mi-ci_d4cFOfW_wz5Orc4yPR45hRHqGdBzYH6ppLfIDCkFrvLSfwYDGY49NZehzhQ3Jj-8llrzTDGPfIJTATJYIZc8EZNrU5XyYIIKtnBJZcSIQ4nsIxX0SwCH5lXvbNY0DDcTsJq8slVuV5_nIOFlKoOtiSjWv7JGRK58AF0akPAwLWoshwkyAqwJS7OeiYFr1NUB1FeI-eCxKjC2_jQXOpgAOZ72RjJIWYc_pT5zIhIqmow6rzK6KuBMyGuHGCWuGdIsOVN2PGGj6lqwLwnFqFuwM13xWyRWuqgWs8QOS8j-ttpjajCSnVAOVTwWOODY0iySerz0TqUZsNmMnfWUSIf5XDl68LxcpI9TulmC2GSle7PemfwYzOjmUbNUyxsl26KG4J2CT43pqu7uhl8oL1xe9hrJfbNzbw8BN4i6GtX2ygeF-N8roQxLeuRpz2RKa5K20hmejDW99sPhkslL2Yw6gF_aLNHR684TzMKx8fqsvVP439BHng_npTsMGKFpNsuzygNSxvwXId47UVyFziQugtMqIdAS2l9sjS33kE3Is7mhaaBEY5fKr1_XUFjVkSS1D3Qp2IGnopKWTGyOQLlsApCdtTV-hmhsdJUosA_mZiUgiukSGpLn6z176vZxHqOLguVpMWIqjM5ExewZMaHEh-IhKvW5PbINMiScQSRujqv-4qMySZ25ldod1A-Z3jZ6TDp-PF77Vz_StdbjHonubbhqlEQ1UfngDKIZ8QqI3iGMftEru09nW7uRVJJKZhhcVkLlycJ6JKN86J2CabxrpyQ4S57rhvNxeNtE5N-xgnBx_oSgMi7CaLrr3UmBy5IXW3WdQk-CC9R70PqHkkvqW-0OmElldGPn_riZi_lq2_qaax-VjomWn4FQPMBD1wPEtGumtXNfrMidKmDtyw-WROdkLbE0KXmCgU9Q9z7lQNZ3PS4T_c36QmqNa2KRfKZBT2qvO4BztqYqMKBUxIf4vqR9urnZohbx73MNqmW-gYYZXJqbmfs7LN9qG1lNPXssVoXbE-XoKZR8gJ1WKUA22IMgr7EULLB2sf6IOaB5urgGUhXjbCY-hWEMII9EQu3yvUkXJXgCvTJUqWlkOljEXd2Qoq5u4oFY9enDCas92bbW2zxVP74l6qZ2ZdLzf_7M56En-SLwnYVODk8zJLUSYW4qH_dq8CQAksbTxqGG1ywn919bNWHSkd1NzhPRwRprYxNM14_WUHuwL-4fjn6aQZd2fNKos0NJ84cKvCJq9_KoXjH7UFOb399rnnroTaPLwctEzeRTD-vf1kduCpbZMwh63iBLlG8z4gcb2PjEbRlioxXonMx2kg-OgMVt2H2hbsO9NscSsV_oWeP4N7l4gRxk7PjGx_VklhfyoWYidjEPCyjgut6TqrKGFTZBopQzTs9BAoXRP4vM_G28Z7meTFQ7nhYuuHSebCHrVhOBdWgXHg19WsDvT9QFZF6T8qwlnv6Np7N1bHz4vWxflvHP6WPneU8A_75LQ9wroDAMrlK0HWmjym9hLOsFMlvjg0PP6EX14Mq_24T_UXxe6nUUm0p9ChNr2drQflGASrh9NAqrkeBWcNJ4cebnDY41nBfH-CrkI9e1NyVjjtq-AwGzDZSbTX4JkDBO7SxlXHgn4DdH1fAlXYzGHyazRoge3o_N-lTfrrxQta567sy-eARFzfprxrnfJmfOEYb6v35bKfJp8JRUQhZMoHd8xLRbu73tmJdJFfg4oW-KwBZYS0tpNzMoJsjvvGq8h6G1gTjieFr3sSr16wdWhBcmPGuBEvhd1i3Qap7-XRPT5IEkgg4BUbYJTjphbzVXLHVkf4vmWNuT1dR3qneDYGL8HYED5iQ8_1X1Ff9TJt8d9j5aIT-8OP6IUnRbop7wNGK2ru0MkaOQpzZ74AKTPKdKNEXUymJdgceQEFHQKLlhc15kMeQTDXkj4r4JhhPeMhSU6fidX_G2siMA_QQiBfgR5FrZa2mm5CXuR9VNteTWeGx-B-NU74kmsJ375m7R9toIZbdL-mB2_njibkfaJV1LGxnmoJeuojhoPBpy7dQ7FEJvOp6S201AQLsv4V7Gktcbe1feH3AOmWdMFZzQDkfFcQdKd2FWzz0w5bDRKZdr4dv7enMXTLsKuL_XGnFdIDcl2e9M8NPTSmQ_6sL1WvuykVFik_xYhIwj2OWy7MRR1chWHYSuV8srD_Q4FM2-yyqnvS-8mmC-6klgOeH3EglaZ0NZelMCEC6EDu0_C56LYZULbtua0WNgboaA9TypZCCVhun9pCD3az'
task13_decrypted = fernet.decrypt(task13_encrypted).decode()

task13_pt2_encrypted=b'gAAAAABkmbBPbguvqtRsXhe0vSRek3mvees_nWK1zO0GE5HLgHZMCmw8e4Nc5Sn0QaOSdcZaiA40crErlbKLHXkbqJKXw-oIbRsa4ESn5V6xuctWpYbh8pz-oJizn3x2EVZmZpJz1GWRtsRRd3M08ENoY4ytMIJXPMEHZ1eUDJPJ-PdlyCm3YV7VgWP_0gTrTFuZwaL-JrpopzqtGF5W_CSEGeBar3k3G3Rj60cltCIoD9Ro1Dwry5v7E8gV6q6W01f7KjMonmhoLN8nHMorzsnhChrzSjIa1lVEZEjeDt26l6KDbaow6IRzHCS7d9tPD2NghJeomfxstiKSGfrRafq0GE-k8x_dX3Ye9No4pJvylPlyE9Dpk-3pPFmHGvRiPrJfk3nPWr1LG5Gug6xANGUC3TjW00eSocN1FKGRvjo3ORTAEsmpF1U8RF46YPodwzbYPmDp4Ut4iRnMZBHNqtL82CLBDSR-cBZgxJHh0vnZTGXoH0A9nXFv3Kj9rJXAyoHFwp9q15SyoNSOil8ufuuEflMOGX45qI7A8cdDS4ISe3Fjm5Tuz9JrZbSl-JlZ8ZGR7npyMNvfOY-JUw9lg06T4YhFirZYpsW65FBALcPWki9h2A3uVg9Yc8tV4P2a5_cCvg6hxceZEauJ5ptA_ws1lCcghYi7wVLTlwxsBTOKDEc-tKVuh8ZMnAm1a9gGoQqfv5jXdEbOALnVqJEY_yyfaTGoZX2ygSOzny8RMxX6x1q5YnUY9EFthR8Vf0T9OuTTaozJkSCgkFcvjNCLFLwq41MEvaVZM-wawzNOjApDFA8DbUAUDkQsphrj3O4RPRIviNnuJohbThZ9HrX97aPFkVFMJ5UqWudbvnU0lTTSgp_2QtVoR7kbodkRPplBChHjztRonbSzIb9dd-0Lo-Z7C20JeCDgIttSxx6U9XRDZV-vHPZyWWUR9Cr80p8QueM9UdlTe23PyFdWwyP_9BSmHLabs53yv7N62cPBtFhPWCGDlLqR9akq66aSBbVIWVcqtCA-Tvx0sPB3kQfBhwGdk2js1US3p5UZnO0NxoEUYihsHuSxs9j52CJKiFNNt5Alh0ADdVkpoPQ76QFkLQKcLMREJORJvtpv3zwzEqBuAtfdZgGdk6KXHwT9JgT37r_IJ0jxzTXUbfKzEKDR4t40Ds52O1zFKCePQezQZfMaD6iFMyPJfh9Nx00AKM7UbmlxJXN1jQAO0DmoNIFdcmJpPuS7Q3tuo6JuYMU7-bIlshE467WlF_Ng_5CxLv4qUQs-dKolpsKMxQzTMdpjoogMhd0UjWLdLue2w8S0Hf_PVqCZL36Fw7_SXt_Xeev5kopUZrnaMhp3o7Yqbul7OwoonYYc1OsatTfT8AoHMLbrqOsP9HQX8PPnLXoYXk58zmnXB0x8w3Bz_HSBkHzEaopyXDBnV_bqmy7E0K__T7mYaV2iaoyFob02abIdCHLcNxRYptHljUlPYqb8X5rOP_65qG9pWLCj5gfP_I9o94OCljSSBQR07--t1JXrD2iwxw4UdAu2bm-eSLq28adRjheojnN5j8y86tUZG27ZSEl0kAhnAY3hnpa9Tthu9nCo02QwS1QxefrBQENsnomeKEXtPtzVQckndN96Rr8Wh9HjCdyuX64BsFT6LhAXGffccSOPsA-Dx_r6VUKFgqESPbREC7c8Gu-hIVr6UeDPv2a-MBX2bRAMxq8VGNcBAdamGyQvD8GgNKRMqS6x1EcxP6IYHUtgCHT16B51h8LdnDyolARdWBepfkMyxbYhdfrIlb2_3fR6m57xRogEBpuCM-MiwjyJkEAR01_MTtCkBRHjs64S2lfxo6sAeIbURmlFOyEq-C2ZnEw5X6aQteibEHYl234xORyuCc5hEemQrjSpUhdMZ657TtDoQN_o1LJLBBuCJ22JWIZNNkHBwY2ssqYvYZB-JBwMqOyQoP1VX3URFgEdZpGG3XC4l7vQw4_UAos-xkJ-lUPYC2rRZQVG9m7m0ctVZ_Pl2LKOiXhK8uir7i1f0Wf5h3XM_9LsIer7HYE_n1dqcZ6Pn0r2ROsKSe4z0wV8skciegUJ9edYyUJ9P-xYMSTQXiJOrn1xQcl64KrSP75BhDXxNi0xuZ9GZqAX1Rs3EiTK9-GwHD7ya66GADChbnEGY-b-HJIrhcQPGCoaunYViEC4vz2FuNvefy_NkUAkDhSfCmFDVxMmHNtpnjS_z3-cL-tn0rJxTxzLnpEeHEeTjVTC9R3E3QQBzMssbH7cHfO3EAXwV47zyw5EMwz7aQdYtOMa98TZh7ZMHSRMfcVVUO2us_bQ0K1ksS2oX--EmJrNb5B7lBtoJ2OntBdFjqKw1Qh_PenGnv4FTAowxXwQg3_v3vEFo5HynsnbAMCsAtGhGSd2bUr_ZIERhr5UxmapANVdm67fDDUk0Tw_N4awd62vSZloDzPUDNFroBzxw3a92o24GyhHbnoZ_LUTq-wkv9kCWHWCeUZnOodV0QhKwUcJokQf1QapOqJX4YQuEMo9hQovo1Xgzb02Q2a5zFZEWOe2ggeYF0qjLQ0m_w5GPh5O863n7TTbBmya4gT8R97YCggqxnnnr8y4EbRUFf1o0-6blzIevaYLEzOmov-DwGoYjSGxm43T0jga0YuZCzi4aWDD8cBiZAKwsOeFr7S6jDwkiIjdg-7eI23ahPBblEN4H33uedseWO3PmxRPLMKB-cDsN7A5_p2r4PVUvA1lvm3EVnDGKLc1MUId9hiJfTQ5z5ei2u6mmvS3iUMZnAT0oqoTRZJBe1IigmPw4wUkq1SPvsztjab6gyrYNSfZngS_g9qG94qerA-COsfrs0XZK7HTHPqIJFAC2oi5UVna3ysSdqAMve0238EO3HbDEpkDk26mIor0ojSlY0cBq3v6td0s7kSJ-u7pqPFYBw5kUokH9SopPDavTI7hcISDbLRNqzz2iqbPR53r_0CNiEvIURpAhgz3R1-kL9A54S6qydH74Pxgzt6vcWl-KGyYfKMCEpcqTzZVfKoii2v4UBQz_xl_3oAxv7cEHtwAUSYtOjdk6UfgZetEGyIrRTFQODatDTKyfMjS2LVEY4m6JxIlzJy1iLJJE_Z8oUqWvl1rv1AU1HBilz5pFFWHmLxtOVhZ_yW0-I7CE6gnI1N5MJEt-okIr7_Xp6w-EtVR07iq9lUq7y9NbN0NNaQ5PNuNZSjDrNwN2uRWyK_hWFcAKya4CzmzFmNtcyWr2MbnYBqpuda5z8D9Tr2vfNfDjRAsUzHL1rjOh75429RhN43JDYTo3-BaaP4vhzstdkDiqFBcIEQI4-PVr2XlE_-jXgEOlOQpCzRswlc4VcoxfY38huJftUkaZ3T7Sf1_exydyBlziCZx7Aky02xgxNrEzJ140bwAPWQ_JjovbhDdAVUThEHRY0FbjdR2jX-kRViDHavDq3AZyu9lqaI2fIdiFnBX2weV-vEcQDm7ZCjtZ9KWPfwkdjrJs6Dtg0qubw_WzyAr4MBj6aZyTJYPuHECojyGI8-HwqmJ_wOMcXGHL58a6VsNOWAehWFh6JQF0ox-dyx8zfXZZWj1ykX_EieU0vc6UWubqUZsuq03I0Dkj5yKaqgEB_Hsa_eagcNolBRbNtH2ODp6iVfIk6C0eUxJulmdq596LKaGj_trFurqAYSJeFSsMOfoVaAFAXHVLr0uqxsh9ngsoI9lcRprbVd-CZCheVjTfIWm2VQapGF2gDjWthKgw60_uPrRxPWubi3oUvEYOhxTNkOFBOa6i7lqf-RW7VgJiu3Aa_8Vmo5nGUH3lFKOH6mFnLEL0rgjzQBqPo20cy4hC7eshQ_WOdN7roTUp9aKi_H6-2af_4KAQsy3nAnj1EIwSgpQYcx7JYMLYVdlYB2kTzozPSEwYONMpIJsp3HRW7lhyKtCdncdbUJGsOtvMeODiFAvazyjy_0rJxXrJbc3bR6xj7IxHO7SQDgE9P3mNd1bfr6hE2dUSZcXBNeW3qzdsUc2YMnd1cQr1u1i3vemq1LkckeYnaSYhH3v_cuWwgg8BRIKZxga3GEJWe5fN2w7jSUlfqPm9bgMKEV2IMshDYUmjKYKeTu7ikBTTe7xiCXnkRDBw7Typ00-jjU0-Aj4D2-0YssZ97-CjuJO1F4hF04TntINQY7pw8-ChNJdW6zXuSJWuA-VFkhn4CNrHn7Oc-Q-mx74p82Xt8joW2wVfYYljzUayDc8-NfXYG_RojPxwrKYMZeGPOtlWiUkKF_lPhu6xK26zIXt5bJz5flsOeZOrPzqhArhTfAScbtKQwb_jFnywWX7ji_7nfTmW8GzfQNWgGXa51__GKJFezM8UcWjJNlYkAUlKdqoLe_HbrJMGaOadKADm9LNv6Sw2JnNbSzBAp5sQsuyaBTao3sNICUqpLGNiiyN6pXbhjJizEuqQsRsD3vvI8k3no7WYVUMZ2RCiapJpwep03utDFs_MmkvgDkUqGjz5lQBqDvDiy-dHhmQdgqveols_oCELKoD0yLeGzy3T44wmt5vgEYOMvvqHaciOSLzLIHSnl-Y_mflH24H0B-LzRavW9rjTZWOCq9yEtJwvuoP4PXcKj-nWS8w1uzIW9p0D825GBhT_i7w_aXcnprJ8i4jTvyka7Kfvz21Y3lnl6TQ_KnlA-gRtGu7LHHh3wScJ2dsodAGefOdwEPYIQWQEfCq0UDoi4IZN83Z0jJNZZ37bCuUnPM3tm9vzZPDjQgp8UyGuoJDypr2WLAmSdO_EdczcZsHtBHRT-W4G925w2ZOTZ1QoDL3ueSTfo7ZuNLHIdyMfU3Gzi78561Zya9IWrLIGLS1Umhj1JnJSfyYd2lG7t4AgfBYaZJ0YIYqMFpx7eq33DC4djvT80SrhWl7ZSUYg6z5lU_dSjDHGDRiQcGZegRWo2lWJZ_BO29YKCTfkg0INeoGMnbGTXh_17Sul9ZsutvVuxQ1g60BDqpJqsbyazLHFS1S7ScqBPQ9jhIqavP7fI3m3zpwv1_81BzeJ0s-ygqx40vlBnLipsklsmRl8nC35TYekbd6QtNfabe8IQnANvNYOBL5Dy1YJdKLBUEmOzMGAxJN-c4lzfPdDclW9To8hb2bU4AwszJ8R_md3RwVjscS'
task13_pt2_decrypted = fernet.decrypt(task13_pt2_encrypted).decode()

print(task13_decrypted)
print(task13_pt2_decrypted)

**(15) Python task:**

In [ ]:
task15_encrypted=b'gAAAAABkmbC-nZz4VwEAzRl0d9S8qL9NXNF0tQ058XnT-n7pVZ6sqKWtiXG2-S3VuGN5Nfn6noKGwryHD-005q-qQMIdY5xvpJGaFIIPlf_zf5geR1jzw2fvmX-A8MMekB5THWvnQlerKkPBmC_jAdka_Ujfwe_ckJ6gyHDG1uRGtEUnSoIYyo07U40MPARyej7zx68xkXNvxKOIzSgLt_E_Cx6cjeCxKuqmW-MVQ4hn___wTDc7LZ6Uabdfr-SyvMUfOFulTz1SMwCTOyQIEFxjKhU61L1tOeZspsiEv-Nn0IpiWNOTq3la4tw35v1ivPi6qbxkmaxL8cUS0wZyztzbHBoSAg9cZa5zbLnCoAX9i0eCU957zukBA8ve3KNK9s8Of_AZBXlzTja_GVTMxiQ_XQ7DSb-pP3qr8AgMEeNRI844ofHImHeX6J1k_ZB0s5MZkk6D60_dTTDUxsrK3abXr_9DJAf1PIKF2Y2ylD5n2I1ZIf0K3IHmOvtS2veX7xKEVS0zx0levsLDySawWMr9FBe-9QDV-AgbB3HSMoSihzNcp4usew9j-dLfjfQRrWmM1Zj82gU4V9rlpcbbA3ymysU75o_8w5kdK8liihMpstfeKDRunmtv8jRRNHChpJsLll_vLt681wL5x3ccRb1cWy69-bwhSZFmqN_zZt5Ps7Z7BwXL8rdVOKF2QtGHi5rV0Vr2SGDPocxIjE651lC96BgfRNfPCTyifv_BHN5RHbSoUgMF55me-yd1-k28ZTRGHVlksghWSdvnN0FUqieiwCyvO3cVeoGdK_ILnIISBx37pKGo-dZhWXUD3hEGoDE0LvZ6RrhImQet4kthhNA1kK9fGIJqY_WpcqSMNoNj-3XwTA2xUAVUVR23OSRhLojpJv0ussNvou4d6Be9U2poG4dU8EkVt_2l45bGWBE0KNs9kVbuwd7PH5FIW3o0jPUAK26xjgzREiERISWPvnme5rKcDZrHv4awxwOZpDc_Z1GHV-EbxwwlPknS5GLsH6z9Dwe_0IJEuq236qFdwBNQ1Ofv7zeTs1u02W6LsqxkW-qDq2dMRXORcXhz85MZqkOPKFvR8V7uD3dmwmruZAmLmRgIAd90AniR-uHEGAuL5Hz4c1amZ4mQyVyQwuAabmWYiP662xkKcjmCmmQZi1et5CLAEGpnIn2H2JQC_HUptN80SDh2mhN7it8FRbPIStGjpc6J0U5wI-yY9hDvy3xwlAoOQJYxiYxDjwYctNMPQtenVBzgk1zSCXoDqv_0Msl8qAaqNErDBHZj5ahPtaJC8xFconINYiUDTIMdYFpXlKp7dZ4HYcEMg_HREtn24RZmKsZ-Vv-a7qS-0UeydH4qinSEZM5EssdSxSCKDxQ1o3jxH8VbDzUdHgEtIQa0h4LJ82VWMj3HK282I1kvL4AyQ8vM_zPH4TpshNpy4EfoOaREF2tpFnFOkJPhPPKDTA4kWoxKYrEEQA1Q5aPWZjmzq-MtkGUut6HIvunhK_KfV4uJHG-97DB5DdGyhU1G_-jetVHsjHrDG3yKlC1VR4UAZAb3kzeINXpJtR0xRyPkDwDKfMVKMta8U6LPFVResEiNyz40-3YP3DaamFH0ucHbnGggXf3IVG-owmKtAey_E1wlDumgpS3I3cxKCMStzk08ce3PpBTE'
task15_decrypted = fernet.decrypt(task15_encrypted).decode()

print(task15_decrypted)

**(16) Python task:**

In [ ]:
task16_encrypted=b'gAAAAABkmtlrp6BY8c1YFSRzkwVb115U_6HJznWmNgcytWll2DrxEeqkTNwl1uj0RAWyTyY15KFCRNymxawqB7k_jB-sFNAiUOnGN8rKHaU4k_qAoswD15UaUgXU1LZBmwn1kJCbnY1Vlk5drm7AKYsM7oAQtGhr7KOkxsGD6FH1NEcUmAtlZlfMdhZiglkxEz0dbO_51RXKAJ4NRfTFH_QFQL0tdVw0RQEF11PbOqpBGd6F4BQfxeqaM4sX-GZqUTbydPJBrI2hICEhYVvh_hsL4VSxv2KSRr4WavOyELHjHUIUaDojkcsn9tJkcPhaBQejqK3nymAltnZYdwmEwZdkL0UZ7h9cHbPbJEbIi_y1Dz3314j9HxKiqT6KPiRMLU8rVyRrB80guItd1xH06ZxEBsJGYH2CaFFAMKRd6jwg2FUd1CAg7ZY9zpei44mVU4BaJEEpw3zDofE10ZfbZ3VLEk0UxHZeARcbRIaCdn2tnRleX7VZQFBlxqjs5ITCFCOcffJE8KfkYWUBhbXd_dbCEDT3OEtI_hKMuFXtIo4nS2P3u9USk3qX8IcMBLdi5yg2nTTUKQltdIPjfBy9HPQ0WmGe2CMV5l5b-EH7Z3Y_oVcot9ax2fjSpqW4Cq1cBc3UZ-fWDQ8Val1-VNt1bCjyClEzz0xkZQ5G4pmEWSnXHvYOuncTNgy6EA3mki4IMZemOH-fv7j7_Q_f11gJhpoHDUazTTQDloTKNfvMaaW0BC8z7aNAko-DgbAPty-J8g2KUgVSvw4dOA8jRngHN9Iv8qxHj5AI8G9m8NhTZ2YUcw7qDqaifiUd2LbmfFU115qT_e20Wh6mgy4A3ibKmH_wxcoGPUAqAlHIMPhgIEivemQ24osec_wq2pN0TWYTvkpk3wmZnpHEHJHtEdEN6PtBKz2Vb41SyHywzPn5eEyiDLzLmbxF7Q1YUUbhn97wuh3XejlnamgG6QXz_vbIjhXHYy-MuEcRA6UKwtzGxKO2fPO8N0EPmgiui1zhjalWfwZN2K2z_LPkh03TpiAjF84HMFCWov43fPAMjMrgXObCv3NWosnIUM1XZVylSt7NNsj1_FyK_dg4SFLcqJLZbUNAXIl2-GO19Clp8Ee2GzY3tLbvFelD6_4zpDv6HxbPVVKkuIGnzXI4AneBnLXSSYxvBSpJGXvtAKZC8BeYUctlm_1sJxo7_JY11MzfkVuf9rHPkSgQFVmlOksnlzLslHc24GoDSpTfUv9xMThzY-wIsOz8-sp0nBlFKF_D_BheuivBUnWv7p8yDHY1dnNW4ZNz8Md6jBXEV9pS_ynSCG0NrY2jxyn5ItKF9l1mfVxRlxMnwkUGO5SujKtlyDy3o_7y-DCpi9bohPvoraiPSPJLizvkqjHkS7671OakUuFchdHBX7oi-MEt2fVH6u6AezN_PcUoECx1SEPDvkDqyPGfTuN7ICJm297EI8bSkM-y_TGIaXxXCXAk7J6TR4cjIh5WGm7l-M0N7JuiGwvStSY7GZ7W5tl4dBMbtX8GQnKaJcwtluM3YH3iZdKtJkxQtmnatSG6sTHU6SmUOPA5Jah84SgVMzVLBrDItxNwIoj-DXu_O91yfVlqV94Ms-EWvFv8q4aGBEVfWkKuak3xkKgqXGIH6wVL1T-M835zeOv6ze6k0YSWdZ3fAYVwiPWOQxJ76zaeaUn6dSgQ2QM3t1y7pItd6rpud2scpkG3nRivk_LJULgSUvI9iqeVNbK8h0YyEoKQmDUe0J8bU9HptVTL3yFDjW-IiqoZNhlOSXJ6gF6fzyoh7Vsb-XCuijDoTSNO-6CfVVEmDDsqjSGGYgZBcHEOYkpdQq0DbW3DWXlHKL1fcABGJMU6z2KcxUyoJ3wZgeecI5hqr7XtmgoBVtv7uohaggHzDIIJm8ufD4OVw3uh-7xF6q7-6w4Q4_SffxZ5Q0TcVqIHT99jKuN-pdzOvodO1_tsrPkPTCfQmCwTnEtkKPU-8052vUWniYD1L-znfTp4srjEUYAWan7uPu0w-lquRCEEr8RRjQR-y2EBY7WtZFCKqiEaDCUKC0_GjEPMgJ5KY5jT0kJppgYTDrAjzg_QahJHnXgCzi4OkEgGMg96zfbtOy4eDLzKab58HQILz7AR_-qZvY9KdfMFOb1O-Heb-1-m2TpIXsKkZe-XOh7MNTfg62CHx_2AUywgrYTzLeQXFlfuPU7-O9yi6COGIANXK1t6lSzNzHCPsPfhrYq78_vwlmONfk7HMq-diktSYaEDlQbwUUcJ6ENKZ9sDOBZICaWg5EfcjESQ9VaJwUInhtYqzvCMsZaOiXVEkeVBNezELN17WPgfkZw1BBzm2UJtBO3YtS4HvtzayT6UrOtqB5onlJDfPjU6gszvm217-oN8fj22VfJ-8v4wVpifcE2Y-j0wFEuJvGSOWhUhocLcH75kzta4mvwhtf7EScnGS2zpLTLjGwdQKFZvJ_xYDotdEt80lZzHxxBjD5cf71dpzHk6xouHUpbRVFMCSldnJsCfrqARMq6VueDpJBlG9JProbL31mKdfJmiOQNKwiXJ1-rwAErqoBbZxngt70UEBbJwWvLgji-PBLhN47fnyfhRWdKggTs5bgCtiOtwDpNNOsLMHzfT_n3UFb8b6EYH25Z6HhcGPPvnetYUgzbMJnnEEuNTbv4uZoWKk0twLl2sMI3NLLJtLgdjbzHmlmNB0Q7smd6yaJsFxdtI_ZRVoCpjTkk1E-W0DiVUYWZAd1YWKVKdRCeDMo7nD1NNO0VdIfzkrrrjjbuScWM1n5gF95Di7zq5hMqy2_rDKojc0aZmM4yaMG9JwNyNSbbOyyX-__qLqXDmogZBpnrQdJ8rGdA7v1JVVQnaikd6RFQUbz3n9DIplLAFxwSHoSlRdoWmIfXkSUXKZVrhPMXxXRg_zGS3ABki2BUEPvQWGQrytUurEVuZb61jX7u3vWpYiCaDCueddWUHb33fsT-w9pj-OAExTwUlGIEMPd49vQvo9aRmRIQr1LypSHgUPSD1gCTLOry8Gr0B0x6YotVUnn-OHYVXkdY-PoA3D-vJoW2GNdmSuw_bxBvgspnooFL5Et_zT7uwcAyFtDVY8WzBVvDwqIhgv6iyxcPuMAXfhItYdJinGLQQMcNEGQuhfSkvK2XxXaIA1eJMQ2AXHtoNTZeqyxmAUn-nP2RVNeGI_cAvGnmGN-vquHPpTh-OBDEJ5DJstLnuSwkSZWxDO8SpDjYvMrIUWRbvDnA04_SFZTSju4OR372sbnVpGZwSy3-rj3xk7QJQGepj12PRLAkbg8WgrKFvna80hTeDqJNy_k9S2edd6LDZVMnA1F235V2GTK6GAzidED5RhcXuBm0QinB_2rABzS_636tiLTsZeIFv6O4qTqEsjFrIGWKgyxJOdCVvgWEBk6IuU5xSgQu5IpaZQk7I614YPR_JB0P1JeBenphNyyMAMvRyALbc4VjhNP8m258yCjZ753P2PrHcP1NeFD_Efy-8Wk3zzDTkZi_7C9r8ng_YMor-3YQN7-JUp-V1iA5KahyXAFVwQxViE_FlBM4RhyXyLHEBUIpNpm60Mmz9P6m07OYyh5_0a1OfYjvA9y-W_DXk117qGe5YPAVBBw5etWDxjp5T8gem9afBtM5GkRUFrMQJv4iJ5T1FSSEbdQNdrYjwsPp1LmH5zY9KX4b8xlUqluO0ZNs11ruazm_1JiZWqT9h6hDPN4LMGxSu-vd_4HSMA-twZMOco7L8jDui5Ar-9QU45roQ3ZOR4P5vXs0uyUohdQgb-gXhzpKh9S5BKAzXKJYrNPQhmdol8q5Tqpf423MdS_j1vpkGdz95VWiCkQFPt3a7wwLDnSTfn0nwX2EV-1s-qu4uMA1HrRMmmZjKdWs0x9lNWpE69MHNm5tOj_2JV4tNp70xy5hg-Q-KMru10-aDPrdKMTGSuQU5gyQ5rI-WyHsHKyXttXqPYv4efNOs5oUrpBAy_1DCu8S_I2NBuPTwE5NvjXkWwHWFZ1KXVFPe2Lc-aaPEFqLwO-Zbf85Km7DLpS2r1PsUHiBDuIEo-Pb2SRyNhFfEz00zkXXmkwR6NocT3kCX4aFPu8nt2becRrRRBJoj-Ljtlp4U3-meae894j5SrnTH1ar3MIxi1Qz-z_HIy3tZ2MKwwfVqD2x233Bf979X-hTjea60ZWsxC-jNQNzAL_vZ2sTi2EcfeYXNE3Hmz9BY5DYo6v-jCbEz'
task16_decrypted = fernet.decrypt(task16_encrypted).decode()

print(task16_decrypted)


**(18) Python task:**

In [ ]:
task18_encrypted=b'gAAAAABkmbQ7tDDO9F6dLyJyIHQEFw5bBHKmF1KnSHijKBVRFrvFS20t3kMt8Rjiq0xc7l5CJoktg4zjGVGC28lV47ZNqF2ZSPVkoN5PxuXyxCRxKALWPxWqxGT5SxBmnf5zYFpM8agvmJNb-qwMTzTAdD36cu1uMkSJB1Zp0iYE1pPUecPq-JXJ3MicMUfeOMRQHUDjuddoKZ4S4d0F6ISrrnmXFGT7xeEl8CVD4UQ2qteJoegdg-9Dl-R5ks6eCSwT-3MRGA5lN_EZIw6IOMtf0pBg-jye8ZJEToyUTNkknyis3hexxKY5rsoeseh8X5aSumkTrR8thiiMnN0zBFQycD8XDqUWDxIw92Vn07u2UeuhXg_m5dvA3ZZnftx1LGtDpt-a8GYqFrR7cmwdAWCmwBtn5UHYpFm0df2tg3gUsKr1IBCIxVecwtVTVTyiZfL0g2qfNg6icLxBhYMIsklwY_SkjUTdWlyPq5bZdZOqexAIUhjE5_uki8BoahkunxI0k77gTofMqliONOwTnc14V1E7BEK9HQHpg2RgebMDssA-UrCSb_v_vUhRoFDu0hESBe5qQ-xA2YLMPirvs4NtQfMqC0v6Hrdha7IhgotUNcuq-2Z2UaO_YvvGTP8BMiwpszfwRGHOFiXOpcENkC9ueKV0Dnquf9wSMi8Dz4L_N-SWwqwp1kXyPeY6UhVemGO3Rwfmk8nlmO7qTBw1vwaJ0FmH6WMGJtwCpcaVGx-GzKOhrf5_Zo3r0RYcw5rZz-3m8C06zXiO'
task18_decrypted = fernet.decrypt(task18_encrypted).decode()

print(task18_decrypted)

**(19) Python task:**

In [ ]:
task19_encrypted=b'gAAAAABkmbTCF5JAnfvVku6Igsp3exFX9_auLdEwtoIvGWWxR-1TdxxyX8jgCO4SVWwAaTMuBkKI1q5SSQ7gLmhLVyF3yf5tan73CCSM3rBRvLmHY6wkP4TCL9zaGl8wX6dz5swGHJvhgK-tYC3Zvwsl6LqKruJCJGC9tr_F77RTgkTT0nxkhrL_fusFG89zVPC73RHezCLMb7p8LyKnkevyPAL04UrbgQFlbpI935eUWg_t7IeijS7Mii2Q1WhakUx_x6da4ipVL0CRA7l4KI0_1VAOs3cXwDav12wD0DcZFXxPnLkw7sFWfEdhVuCDZHRGQbHm6JusfndqT6Asjdn3mlS3SUQpg1s08whIOtP_v5jQCMRm1xgUwOS4zR2w5jILVnL64bgKJVJi_ENBnQqRaTIK405deyDbOFT63PinxOUncbWbF_OfvUmSoKnD4I5xYCrWGRQaEqdSylKmNSHhbjBiAHfvHhUgMkx_G93srQPUZIPxoYyuiQlSsC7LGEa-FILVklcQkVtD_unoiZnvEl9tMNnnactmJOpQ5Lsg3GdQpXQbalEFqDhs2i7mpl7xV5DW0lN261lh6r_vsQAfuNUIbQkMOTjK_1NfkDFG5sFZWOD3f_W_WGblcXP6QURF8YWC_uj_o4bemUUm8t9ddDyAKNjukF5IPkHy9tSstds41b2OBJy7_B3ydn9sroSYzLw75ldXdUmiCh3gzTCYMT8OpEnUUNFeAcQ8QKD6_OsnDyFwsWYhwKgbJUDcs2tevctiAJOumLlK8BYpAuNxxQcEkUOGl45TzmF3oyZRrq1804fgZAvSGn_uPlVnldRNbhEYHoPWIfPsjAwns23qVHcGvVt2cUzzFSYqLEK7zOYjQxp-iH7U9TGp_a-Dr14TUQV8ciDtvCiR_0Ohj_zb7AK5zNCkFZwEjpve8Zhl3cQhk5wTAeK6Xf0_g3hIJ6gjJGMJLel8tclRX1ldHUwCaU210BnVGcAi-YIIiTQViIInyT17g2KwOAfnZeQnLuAZJ7eW4OgEBpPK3OrMDwLev84crV7oOknvlkhSG6-OEwfHcDXyn5go3MqA3GXpVW-ODeFneyKAMEBE4-SJ7RXmuW--Iy0h04MhRTOrtd3p64NQcW0Frz19gnj1nh4R2-96tEOlQmL9bpK5P-n3vKqJ6OxxYvPjb93wQak54m_DhJSVgwH3HNp2ShFnrJlJ_sm_nFN74XNqXe3U3G4lksL118_HnPi0my9NB66KL69IbkLinPRa-O2TvxJQN_jED3fnXCaoApcGfgLk24aHLbOWOzjcvF_50nZKEI_6IbBCp1bFcJdGStK_rI2FWG5r_ubM5BpPqrLtMYdl_5tYTUfY85CcXuVARkyanXqChPlLACgjB99DU2iBQYQyYnLv_dMvjBnTNgdE5XlDykdq-592uLyUtKKZA4pojbSNgpboerprF-L9HtKYceu5YYOq8nmQFOrSqXsoD4tONRwdUL6ae3DVTgq8OmeeGAzdUQOr9D-o30g-VtxwZc9WvSgqOUK8B72-OML-vnLyfNYTgRu0fCa-xTfvFoq2QgQf_VblNZjbNXe-cT55ZGkpjXZKjOLliWnUZfKyjitBvoUNnn4tsBvAWA9L_sScVjmxXdWWnFb4OXgsjh4iyItBEOfdWdjH3IJcpjXgfz7YS_N3vbDFeJOx-dOhGwpcX-MFDivxd4vphLxMHxmq_fcLL_xr7uqjdGrd9AcaCFh8RzS4y8gtcAmIYuGYBFf2K4AAIUsCt3iHUyTJhatNO-dfBfvEh_ZtZTSXVR68o6yLVneqk_GGRKfQ2LjcBre-TBd18lshX7jcYnhYVvKGywSfkVYJY9qv2x2w_oY8vj7mHqRGiIkSpinJf3rlDhKNI1XjYfC4DU9oPwp-NQ-_B4nH_mBqJPhTnyleDl7qvgGoKCKd-CkwdpUwPIvktmytL-oBD4YZZ5s5UGGU8iHyEIOoZC3_xNk5pUQ50c6xELBFawIYfte6KgrsNjluR7SWer6pT201SyWdPjrm4Tek0oGQeMckgc-BLDUeohV3wCuMo5KmF_q-jmzt1qV_R20tIWc-sA8Gjyv_7CI3DjePtnb2Q6W7FzHTgOKsR8kCjYjOEgGpALxS3scCDtMeA4YmxrG-10Ub-4dQR27mumGgBGFBWhgYF6_yhIhC-LdQ-fygRJN4uzU-kaoT5-0a1haYjPX8oY-d7tXlhpTsVqK8R2Edn3p83-nb8FEJ_XaY6eXW2PEhWWsmkyde9DjOytrpjGJT8iRkABnSJsUie8ko_79ndHaNupv-ZdQ3bPGekXTsDHvuivhFwNYUNqYiONTodTiP3GTllqDc5F7awstW8DxGFQe47t6RW82lBs8YUXSBpZNxO8w2l9hmwBBBvcpo58QaV8Km5P08YpwtmHHrBtw9i5ODhlcA7-66WxwUpulzzbvLfSKk66d3EpFbblEubWVDq563k6Xn7Bu-HeYz0ZvGicdaG-DIjYVQk4bTxlTXYElEDPflnYnUA6sCgMHqxe5zc-8--j2Yoix8BXwl-_SeTeIpM1Xz0DBp34PoRPlkdWw8drjOnn9sZnxoRBMPOdPJN4V_jrGHdXpK7cxizVkGEiu_05WVWTMnpgENslQpWH3WuvPBthJMtKEV8s7SWnwu6pc6Gs8wqViiTTzGzes8zURQuPnn4mj_fmuy4T5zVE1lacZ-GsqTUY3e76-bmuOL0Iv5oGDNl4_4MoWAsV0RvWblW8LnnR0ZpNqMA296LuLeeAnzINAj4dEKp9pDIz-qmSbeN-UOpH6cRYFq0zOSSYFZH7m0iL-T31tOqEVWRKMtPg6Q5gjbMXUhRp9L-uVLfJq8lzzYg8qZYO_sKJYjQPVx5Xo0dt_XhY_bPdyeMVnJTalQ1-uRm98I6sxdh56h7y6pKz1R6JKWriOZ8Slt3ZLlmN3vWqi552O5ckI0XsiSmy6pkMGViqyxJsYTwHLXJ7iI12m4nmNMxuboj7iSt7CVlX8X8hZlNYanLxh6to7XeMz2bNLJzs7qwjvo92xFD28tpNJLEBSOnJlKRt_-dTy33CkSSv0MSxQOofs7R1V4RLLUwkGbt2SdeyUWN47lQBBbwx3agyiLd_dowzZvuvd0EEew9DqaN5HINJpeU61jmGy-bp2jZhCUMR2FjEQTZ2SMkTQHCin1I_hiJlAm1IHVR9A9Zu_kn040-pMU8dlzEUorcSmjioACGI1HXnPMHUnQ3KNHFyuYQhnO96kSZa6cGlaBPwgnn1Dm2xwPoIMP3lQxW1euZf6dgRERimwiIoO5YShuokx9KJpDFuNbW3L-A_lZ9kGrFJgzIMwc7VWm2waCXzbDZ_ApCEa0p6sN1cTwJAPTuJdj6I9AJsRml3IQg6P-EWqIy1_M2zexvYDGg8qhoTI8PAP70DM1MMGE0YNAqt2qjfQrFc4='
task19_decrypted = fernet.decrypt(task19_encrypted).decode()

print(task19_decrypted)


**(21) Python task:**

In [ ]:
task21_encrypted=b'gAAAAABkmbVJ5_zhXJOQdcnBs4u3bFBmJ9s3ysFXhb6vweXf-q8C5m96mJlJVe4Wg737rWM0N2ZA7l1-LTy8LYH5x3e5zOQbSISLYPK01oKGdBaxnAI4-toDYwrEIrfvEptknznDOlOiAuGGhtuu3uRX4hhTTs7mBdQCsrCA3vpPe3WAKXKZTzmjRCgkNamGK5_kZwSKFGx-SYX9tea_ScF_MJUT9M9OZOOjb5y92CuEIwVyUVuLxAJc4KCKaiQyvXrjejs4JiL-oNAluBQlX8Y51lx9T9aR7DBzFnlrbJWrWdcth89Dpc1lDzxBzQXsN8GXNUwQos5S0Tyfll4so4J2xqu3k5kcl7SEHDvccIjTCLx95W5124BEU008SY7IdxUfJSfKYzhLw3ZrHRN90Ldd-TwEqU_U0n-QzqtfOI42iD-_KW6PvzkbjsE1Xd2zDlCxpm4GCUQP7jQH258kK-ml2ZKVxQWFgZ9Z_UcESh4FqFLB1BO8_7Zn8mTz1RrEpU1GGh8gfe3yQ4giD4zEkFab7vFQ9snRg4u5ZVkExYmkA0MB40jez1nZ5uy9csTYpnbA5MoFvE9A6P6p77ArtkAItNvp7_MuQ_NFeME5ybTW7JzMeNVr9nBEXyoWyj3ommaN9B7U9MkhGIvImEE-nktQlwr-pW7_e0l2qyMc2fCVA7erDhJT5dM8iNDyj4oKuPZZo4gZkqn7q_Rst4zgf3XcEicPSUZDpwlSHkrxLgip2IeG3xGTxN5P5idKKIhuLdaQ6AP-KqyKopCqbxO0o7if7WtOs6lnClbLPJuaWNPIGQFv908Qo_TjX3XtTOJifHvrnQJ40N0NVJd_h1-4ub7mqnIIT2ErJB_4TP2a9-vKHABRKa_C88h5o1Z5SKFxuaAzfA97DuzutFAnZ-SIHiStmyEBGPIRSqk4tgvqX-iE4iikoZvJ3spVRa-OBDrbgDQrqpZwEPox3-IFRqWGxYBOfNbh0IEZAw0sCclnI5wSuGErqJahdPVqBElqd9mCHh6m3wP64CNXJM3DR-mXXlc9awz6XMX2WPKdFdTwfcb0mrQSBbtRgkQBR2OtO5ya57Yy67HPKOdRKNUHo_i326mb8r-SCXZniMHnyAWAWFRTf0i2KEoafNfR8_7KPueOIqOb_9jDyZrZJRn1lml8DtT9cf-O-LDX7NEDLhSFkxfUvS_PkfvQeZO_QL8MoH-vKk0zv5S1IdHYnaBe1CmKZfZdbYSZHXJemeqaC1gdU14k2lKMDlaqWVnxaP6AB9P-Znt3DX5GFf3juPV_6YsISuO2z9IlleNHE8t48WnLUZUKAsQz_kOtSmK-vu5aSB3_aAZoxiuC4dYhXjDKcRJrSPHeqaGpevBQT05AIr5Yr5nulvRiAfc67cLOwHma1pkMOvHlpaGuQjSJEqQuTclkEF5lbsMW2mzVCGeUWzFBHOoEFvCDkQYifHZNXuuEeNQDbvSh15ELLk5fhDJdKYvnlWESTrOSGbThDpKpoiy4SdUGfTM_n8wSIH1bKB_AgfLVR8ZwS2jRVjDUrkY6lAvo-EQZgrP9AVBUvGbg6f66cECxS4Rg6QqdFR4qzgXkShu1OZqUdeNWJOwmIHx8iOmPHALvwSDiR34iDXVVPIDesJhkUWqZMpVIOdQIYm2UySQ6AJYWncnWzLDZw2H_9mIqSU_61kiPAiuxpScmhKRgtlfbTnVCYI-QzPtVADhMA9W0M_CmwFaJu_Mgp4WhWee7caIpATtsORUH1ayi2y-nZE0hkgJ_kWCxZnulpdduBY-WuSruS90k1o3gb4cIBnHEeLE_aY3x0VvL7QCOcycdgZvh4-bXZDN1ZHTN2f33SU6cDCdAxDjxZTMr-uuMHriXwTnKAlGo8-kd7OQyIHlqW7rjuezfycrmqt6VKPNuu358lQUbfI8fisyGYAMASjvAnVO1msGxBZKv2ILu-yKZfGn-Xm13LtgNU0gFOUUq_yB3MRVEFTulZSd-MSOkkLzx62oUfxsLsNcmxW0zK3WgHsgofm2S90CcSay6NrhhaHUb0AGkkR5I0i0cAUAA3Tl5YLNzOtvoUKaedwOWDOgV70vuAwI8dTCPednBjacks9R5QnyZgO6ntJTdo6eWWXEi83JkkflPW9OCchX6F9ZHmm2oEkm_3Gbfq6nTPVGpmInc-Aepavlp_0V5HlRzYm2PoEuf6IcTmyGld-8jGR1z6babZTYo7tRLboS8QsNWhKSVpNdipP1dZujWyuikImTCYCBwPfc7IxgLD1MwlPUjasHzpySOjyR_caGXpki_Xir95z9BUXhm6cbHXt2SzxZMjUFG6g8K2Lw-IEYzyejFwtOjtF0HC0eU2OYVt3FUGt6W-_blA2eXKF9RISohPB_nkTF4NKZXo-K0AqKSsMTLWlzLF0tygEaSgvBZ8u5KFqxiXi4yoHER-ePAUUARg5C7TVGI0S0FBKmSQ1cCa-hD-V4Y8xNXeWYFkvbOgFieK9emBfTHc18cq0AlrRVInxMteWmicueNWH22XcgVIdB3Ox9_m2wdAAp99-IlmPca0mKsMnKNI2CjD_0dsyy-af87aY91fA61juGK1jL78J9ePMACTXCoxtY7bRrQen4DCyTQT0ifhvVuif3sWcox_pim0QfqC2KJ3mruFEn_0BmeaO6hTqvizkQ2J333WhhQcNmb31DDb_qYK4gCBpVEmx8Z8g5iVBKkE30I97EUtUAEKsyzazQxR-80yyKKO0jCSVyFIHS0rDRsLo5Nj7Am9GNKr4qzVrDlLMiVWCQmXSHZAU2Vf-b1dQb9QUvxhED0nXSwbKliOUSZRHvfe2YJ0ejo6-0iV0ZkT02JbY3TudoaZ8snQy8asb8gGsVtPgPa82F5byAP3KsEB8g8TaMaAZr6KmsYxAm9QQb1NLsTjxcyE4-U6av2jFkBUhe_jKbdV6lnXafamJMDFlFTKZD8yy_eQ6MxlVDBKoOv9Dbhsbhz7nRmIk0UVnLr1ol5fiu8I4AL2vzP8MmKL4jFkjhI2qZIKHEfz4ZBwHzsjbm_6NaobiSuWvntRW6A1KH7a9e-pagyPNYYCIM01PJ1T3IXVOBlErm9G_HHMdOl0EVzP0DsoV_WBLCN1XJEpvZETnQx9OmthDw3XWCvYY__TOIt7c9OGdo2_XK5KG6JLaC3HEHn2um9ggNxFTGAituh0RU40ZJY52j6Kbr31eAI__-9Y9dagv-GfV-BPdaMc9uphW0Gv-xQOBwr2HIKvmxXOcuy-ijvo4Fc_WDPxjR3LZPaUAm4nPmuhFe9G-WBivzIgZPvVVmNIRRnErLQ3Bl1tB4bIdrHI2eTYEXeO-Wz8kMxjgUEyly1JIa0c6sRObW7MQEpaldf7Vhkyj4OwtAhgCI3jB_qbyMTxbEeOSQBGUYrDO8ZSd65P-DFrVNQ4VnniVBSsjzLzPEaylxAFNQCDCZ94sljByW2jBOHmvAz7nUmnffKm3rwT4SBVJWdK24iVd1FQ51ocHKWgzyin0X4uOxBtWpYKO32VJhhGCav59UJA0xCjkYYnkEm8wICZp3Gd_kVtwFG-ixBdh_q4q6XKqJvfw7XFUQBYEhQMVRfjToOOXXZolSy97WRrj9-VA=='
task21_decrypted = fernet.decrypt(task21_encrypted).decode()

print(task21_decrypted)

**(22) Python task:**

In [ ]:
task22_encrypted=b'gAAAAABkmbWYdY1vcTdA_ZRkEofFxq40MeJRVV-HLTm2zin4HJu3ZvV8Ak6MWr9OvyuKuruXKmtSe-dWyVn2Aps8TcskX_UpK_F5PcG55QHWprRLuvBVLiPwJfHtVP2BzawJ6PZYhnfogbn2X8y7PrfBwO7JC8V8fT6tuHAoVpM0ZHTgHcRbaiH6ARoHOACcZkR-bOKT20-Q-9gyuCMgGEFH6ZYo8tHAWPXMj_5md0kGVZ358AAY64lBEVvMqf_IHbwrcH_1EIKoCnyYud_H9BgQBTvcqDE-jWG4SYDz6PAqfdwEAtqv8BRInfs3W97CUG_xRdbsm0tFu8C0gEi5seqDO6y-o_19SDCMuk8_06cI_pcE8QnjGturKov94PZkSKD6znpsWZxATB_1pT1JetgO9Gvas5DAS5yEX3R2oHjfu-Uj2JjlvA9KAmUYIom8E8N9Nt5A0GwQAA-ZOguluLZ7WkVaydFDnaqQcsfEq5HF42XxiR80Jj5DEJhTpWiFXGPrxyiLFCZI3bf1WHXCqlaQWNRiF97G3OC_CIU_2lQ5Z9a5L6sHl5xeRkZfH0zvKS4XpsKiA1uT8iNbP01CVEZD_P7gauPiazXl9UOrooShMmzonj8Uo5TBtiXORXhryO_GlWpampumPe3Wy0SRFcMFMfK7offAjtl2rYNc9rtIuPNsZKoy_qLsefOcV5jjVsNTWJceOGq60M-6qdEQiAaS9bf_maPpjnQ8H-Yj9Kpi6FnYfa0eeezDPPerYZTfl7LB893xcvDB4edw_444FWxh22vOrE-e0xgBfoZ5Nr52y163JuaOvrVhH_jXc8iAeHykPNZM6NOm_Fi0VerZrd3iEc-Z4SCI1qo21VMdlbXXLxCnij4CR7rVPwpWiV1f4jesr6dyYDf2vAWjLWINthShA_fiD0JR7Ch4WWBCUkNMIvDxS8BB_puRf5Io6t1ctGQm74iVHrYxoUQyFaHHCGDipzULvhFGWXsSB1ouV2vVJpiynkyPKib_3Qnwyyfe3wphkc3ZqwIHfhBgAsdxu9zh_BISvG9eP8cOM1-OU-x4r4i86wDNnlECXtDphSMu4voFUfii-itZQqKv_5AmJS3kFyzdfYy-BOzl2AzGePmpv5Ry_uHoRw_XiaZ3SOSKiFmTyMMGhQAOJVo7LW51SUTMAqk-bxqBrskPx3h1qA5EtGVnbFj9iIR-el-v5zC54RLsyZIkG8DFeZRhqiqP57G11OabQoUB4hgFpfxIwwq96LvhzqUcY0SGrBWn8m7PIwU2ETESlYvra-Mz22QO6Me_CJuwS1IoBQ3_qg7_B9jzMmVUn4Dkw0QW_NNh2TGNapMR_NKtOIomktqw-wz3FVshvuQGiFvn_Ma8lJ-O9RRl6PfTLk1HV2aRrB41Bv8LdUIwLB0dcpbNedCrOugsa8ZzTLEfPam6u0E7Ud9e5Fo18rdtVBFf31U3lGOia9TKsXumR18VX4L-fGChSbvSCLJoBR3kuXDHiksH6onNNuaH0jRNxYNzBb1jo_U_xBzJlkYxkzOCgd0NQCe4JUEeun6n9K6KMoeb_bsaEAY8KWobezbH5hZ63ThdYcTOb6PopmZcdi7KA_haUqpey2YJ3hwOlYdcIFd-C60nhjJqa2cxDnGF0nHcLuNB6UKyxOUx-jIs2UeEyKFfu5vHR90q8Vjl256k8-jM1go2UjNaZ5RrcfqaW-p9brnXme01b5quYxFxg3L3gflzxZXLiiisiTyga5LVJK7gM_rBVGxSgfcEy5z_BXDipw1e1yGJEel_ZclskYQzFH_u1y8edBFiMopr1JO-ZzN4lcw9gsZLvNd5NaXbrc_pM4E2ZbZJ6g3fyXoIfj-egLlDL7qpiFi0qVgt9_lG_dj4brlciKtpA7QiWlHPWFhfIejccIUJqBSkLViXe5Z9gAw8vwkrxV8v2xLcXtbNo6AiDSbWD66_sohb1PmW3sDnqT_GIaHmO26I7mrm-o4sqiTNuELTtz9FYw5kFra-cFRLvcA2oFfWNMZhYGsN1h35Gl-FVAfQXiDHsmJuPOeMLlImezbdW4cZl5r2fsx7rQ8QOK9xyGIy55mv_RY3gaolLZ9cL-xUsnTmLJulBN4oeFA50vnDNS2iBkFa6XRxmEHKCvO4aME2aEDeEGJFzLb6G1CGIZeQYGawAMxfEiJq1WhZAi82r09ahtjbZQ5rru8ywTCaLgK05urdftKqX3924n_iPhinXZ31Ty78UDrmuRBYnVDX2F-lmDxqrw33fqo9P9Q-idml0jk11XnOmZzsUP0oloEk7kWmiTozk8pLmDPBROmctH8n-8HgARKXil0NZg2Rlg5L-ddZbRRvKnoV2x1QYoIPJ3LUckZu-XBuMFjmDNIrYcMM23VRpc8AywLzmBftRTIaZlWEb5sq53IpABcuY6nVOaNErLtOwnZzwUASrguaAWY4l05K4wTEyalEOBpGNRK2Woq4hGlHvzKsE2OfId-seIdOv9MrIku7k3R4UuJlOQpWJiYdDCFEqiKhOw_wbM9aMa1zT2XE3nkE6YdOMiC-ytcfMmHDoXAQviVQKIa4xBXZgjB8QXBvvUJ4aHkd7a-y2pQriTQpU8Ix388qo84xBrYwaSJMnpi4MqmZL9orTi8Csq4BB14tE-5gsJpKI_ilvLDiYKr3T9fHgFPNCrAUG7_VQsqYX8FekTH9klm9yg2m6pfeFIEhtflEIqzQZOF25RhYP0_8FfLrpsfnsOE4SaVoHEKp0J2wV5u7sVrWpsvkG3SEGlmKOQY05Q3nLJxy9FlBLyYVbgy6y5dHMCuRxJaKB3VXYD1_Ig0fZbMfiImQFD-hn098nQJ0PTywyOvdBSPhrBYSWZ2mEoAFW59aAKwlShiX7BdDo09SQjyWgiXzsU2gx3kmwCe2AokMs8WJ51E04ZKW3fQTr6_JoXc6WnyV8CIlrC2JebvMs9NkmLIpOVY9JRy5WdxT-WcEDhq879ZIhSlA9936KA7Z2lVUZbZpijS0jh7uBbOeuNZuyw8KX-NC-aMmRHDhEqKDk5Dxz_entCp-DQJ4ARjqtFXoVebJv6k6FhbyPYa5fJsAvQ1oy21LmIRgpDVbYkxEqvsrOcNlBUjGZPQWyfPgp_wzZd-4TRxGFjjEibWDwboXcv_mI_6ch7y30nb13_-y3XC07Upi70HTP7QkwtLoUklsCppSFDN7tOAsbB8XD4EpGskqLHwURpthYjLSm8-3lnSnw-jcrhTrCsx2FyEFAGkk9dOzAjTs3HaFYZkKUjjCNsUfOfMZuWfCTqVMFwFVDFJZDWhW_KHTl2ANB5z6wiCKzWbPWc2t3OSCYcYj4bM0oootcwU8EnyJz8T4o2o5-_6Dkd1XZByUtdXuJmKSHsnvbTTSTWPDfp3CPQZtKFEzF0IDZ2QoPjpTKDNAKZa_kBTs8tetel90IixkC8WXpXeqr3O8gFFBYC9y5aDBYlrsVZuFpHi54K_E0f_k-Bo0-gj0xxegz5qyrKu-4M0rTa3NwIvR97Z2F6qst2xEEY3UXRwSXLpAfAZq3H0Fs0Anqb-jzE2mNZ0lqB3qZUZbSVxw_X4bsNWIrGVeH66fD5IYjTT92jwpTxDvN1iS7sDWY4gtbE-wqGcJcpDMkEVpn8AStwyDywBAX0Ctl9_UrNX12jp7VzlhKkUYpcxXjacy_5ipYHex0jI9LeP-qSsChM2PoZSmF8aObCnmwKdkplg_8TD1qR14oMXRdzWAE25pi3EMWvexv2-HW1ZTe-ydzab6ZlKVlYpdwCRgKg1pxpn-em_NpAbr77YOV2QMiW5s0TWTVE2YkVZOPECEwClA1Kz5xAQnUA5rz6XojsGHqNnIiV6GX7LceolTTmDTgsBl40j61kAi7-vt-Vdf-diYuPO2A4BzG7MnJ6dZkds8ponrfkiRwj7Hksguy-Dg8S2pRMI2mkPKzKqUB27PxSNq0hyQCGTxDuqQsSrnNj5uHkLi02BwcDgF_gz4ezKMXdd0CTqSinx8GY0T6gJUOne8wBbPXVe-WQ2C1VT_6hoo34OMSI7QNkNmdMoaR6YDCMI_lL5EMQ5CuBOctlrsNNVyP1CFLVG9jnCo7h6gitCVFEVEpvtS_VQJW4_xDns2IHLwR5q6JJnt5sblFK-ABeTnCYqvow0XWqLjKyySiHj_ZYTEbILsRa2WChXwknbnR-CjNJkcss046UxQLsbwiu3HYUzc2Gegt0PkR3C6gVvMc-Vns7-grsg9kklHQjo9cjAKSBtivu1oTbbT8wuEZehJG03eisDWjDjMfVsxtsgB2CYGYIU_ODdInpZkC_lRfQNxybiN-XPFArX-dZ38aDVU6DHm1HAxEL9a7wpJAN04M8x73quz9EG5fKb4rTGfcywjoCBWyMCH6jPewROcGxgaQ6GhlaLK5XGBMWQRA8EWvLssU0gneQB70TdgXY7TMpnhyqzKFi-Ufl5eLDJg7ZrJ0sqGuRZkdnG9yRkWdm8kij_bdSMswKHs_p7T8bCqU3hsv6lPJLOoyx_FvT8vaXRzYBYLCnoOFCYkxo2Cpfsjxvq0ahUTOS2J0uRVAZ7ptHG7itBR1zZQW6q19wei36gB_SDakJj32H-K9FAurF-o9VdUoRAW-lVAXGu3-FeMePx5_fjHTRrs4Plsiqva63f4VrgFXj-Hdnn9itjXHnbteHzWsIB0iBtB7W7rscRhS-4EjvFgDv5putNxH25gGnQ='
task22_decrypted = fernet.decrypt(task22_encrypted).decode()

print(task22_decrypted)

**(24 BONUS) Python task:**

In [ ]:
task24_encrypted=b'gAAAAABkmbX6QIzxhq6XFo4S5IJNz4WEb6-tltAgvziggd3K9tJQQ7oyohyxpg-0cZtKHIsjlUSY_RtWNoOFNb8GzrIz8uvCyeZ-olZeolT_I8UYEf-f9u1kA3wjf9nbPaEtOEuVuLA2tw2vzBydQiGn5Cz9zSBD06HNuJ9MUDdBKhMOCNH2zRnPYKChXYaw5s1YPlFEhlXMTwzOBQcYkmQBq5k5GMbDpmMhPM_YIWTLLBxuuZhQMdwbXt3YCm0Vwm1FSBjh--xSCB59-gocZ37KzAqxaQ70OmcoslpbgcbBGsdfQV9FwoV9fDrEpS3qxTcseTnfuAVKFIN-DFGS3hmyQzGhepyEcRK1FzDas1GgAK9GwMmrH1FiGLKtZb0srvmYe7Pw7lWDOWrKjSW5iexmNdkolqTSVHNDWQekn75Do0egBJHI1kikeSrQYD_fTB9QlrL_JYqn0OTPStlkvO9sPq4jvvQ8U9DqtS73K7QxxE1wM68ntKW_mbK3I1X6Sl7QGshV6F8KRxCULQuuz8_ZT9tEkhVxa1abnHQABl8hfT8oQFk53QoK1vorkBL73R6i-cmwdqSXtiwV7BS6aTQfKXKdnvx6T44391eLH1O5HOk0p2zAPWa3umRgN2wwtlrXdYK1gNDsfWhXE9EN_fEB67t4sHe7fB_idK87cHDpy00B1PEKTzHgNru_c-Tzxql11l_Gspsfz2NtOyAsGqRPfWtRzNsebD931TD43Jx_b2CQB8Q8fIV5CJUO3KP59_5J9dm5khrXYT43QT-pW7tmvVGzsyet8bXJHTp8w9_BqXOlqvA1Uyn24YT4J0ei6A5e1DF8gOXhm9f9r1FAsXiWmzQxJ3q4frjjg4EQj6XJVgRH1GGHin6Mh7wzTO3LG1aYqsPY55oJsmk5jhRfT21GyP6H5lvuuRfvG3vFVxPmUN-xyfBOhHN7Qsq6QHs6IZlPNSShAF7EQ659T1GIP2nbE6arphY8jYd2pkkwGR4jOMP1p-KPtrg0CGyamFKlDokhkhqKLP0_s2ne1AutYe9ihHORJjTwrLb4klb9hKPlb84N-TkO4VWPxYAKGZQV8k-6bGA14Y8c5zDMwW4gLX02PG47Swg4u-D_rM5yOQsuU-0ghp8QTK9OvBMPJbcnzq1xUSFq8eNOEX8DA5EOiI7dGL-ssjOERPQDEiPKDNmW1y1QeIouxas2o1FWww1sSt-fiwM4QBYci2EvLRap8lomPk0-KKCQhZIK_uOlmYe0SRWBeVB8FZ5KFREmLbJr9ujy-iII9lbXS1tweJiPEPY98kuk-17Ohn7p_YlAce3pTJkeY0Lnafs2Wt0ViojDjq9QtORqRhDQMjfyvsvK1oFo9bamk0mk1ATSSGKUGroO8rmbuE4DC7dTd7vOp_jUEfAnMJoogR6fCV1XVV-bxkuWR3SxxiJmmtiZ2HcaKPWrkCrwmqJxdS2tfsWt-FPtv9rnoFBymEB87JMDUB7IF1sxW-SdzP6T0lx6d6O6VOpI_Cuu5M1D9-0WRfe_wiWdUAQA-O6qrYD7LaCokDxbUlTK6iEHtwxVyHvtghZpeR3_yOUG5P7IsayJkW1dmQpauSjBQl4gMCfgZNpEb5kKgslRn8yKCi3zuP_-k-HvphytHhWE8ca9rGcNwR-jsHVMDEkwuQUeliCAysYGZjPXC4cLarYe15qYe9cBsyxbLK6DSJ4WwWfQ23AX2WqSSenb1QMgxhFU3YMGFTNLpZ8Jn3pX34ql0wxlThWS13EbZvsnfj72IGP6NC8f0Lk3eKvm0IKqUPWTu9BcWeapHO9rjw7laKD5kPaI76M-D5L8BZsfw1x4MdGu1K0MJ0PsYfNasi-66I3HUHQfENz66BnJwSyDQfDUW9eyW6IghZcVWZBXF7xLPZPCqd4FvwUGVx8oGnU9nlC5wFPzAh1PLiFbfxR1qlynuNfJmpHDErxiY8iHW1CwzZGCjD_T2wlLNHFeLDJvnytoPQwIbLMyFcLdRGahaeOcN9ggK2WEeivVsjWHaX4LFhI77nofcf_p24dmw3UAEwbxjUfEdMD0aGumfyKWStFFRZfPnc9nElj5mpHKnLEzVO9zfJAaJXnTaWh4E54n5kweOwax-5kAxSmZx2znxFbnVWla9WG-P7ye2Ko-v5lMI_gEP-aXLGetd7pkVF_-T9QjYHoKdTJXEq_dN4FHh9_jWZkT7tF1o_zsLub4mZTXxaocWQYoB3JKKFy-HDJfgBUBYSBu-lEGw22x-H_uyV4MW6_hiW1DkSocgU79Tmjs10Fx_2my4tXrsBuB8ABGeQkqKPiA88RbDzoRzXyyK9wqae_I_kf50rmWpY6Oi4NJyCuKf_xPK1nqSc9nk4nf3iXWOspqHoo61U0VReJ-5g5Olr-r78I3OR0PNvs4-GaLsiAV4oPSWKBtWrK1m3sBupDQa7KDfBuJam1NLx_DEIcr8nXOqvIfSJb-6LbAGhK1DI0GEfRMPE169lvehVjof-fMUJZzn0gml54GPRFdayd9L5zhs79bu_yHJLBo7IEl1HUy5rcpH5RdL0VktoMB8AcMXtsRJvJ7SgH-CWGZ5RtmxIQrxnlmoyKUoDEBVM44DtFF2nDu4VtLYGIKbAm4Erya1IINAF1Mkv64rzHX136h4V1w7n46JNkd7iz6kRy5VNyRock2phpgB1pdCXXcqW9rzdU5sZZhGRR4IZRQZiO8vXw2NOQP0NZJVJxNoFO-5y4sLowSASJPWl3kDZFmH23EqrBmQx_Xn5qaaCw7BFBkrJv3aVlW7eZ1oiMWxz50jrt5ZExwHLQHTS3tv6VJkHV96khd4u_iOQSFNLAlJ6WH6X2nU8eG1DOAVIMVWFmK1wNG8Yjn9Xu-24ICyWJsY4GY7f0zI5IOlTe64FSC3W0qkt1lbCSKD9JbyCTf9y4udwUOK09K1leZEh3spxZVpYogfw7FmT7EmYTHFwXn-1qfBP2Rpf6HbryUQXiHLzXx5Jv-fcckK85iB5fUOsWcS7tmmHeAB_ZPtwik0UBVAEO-d68ApI_P1D1PaCv23QI9WIKpmHH-2RwE5D9kh0V9l3uwxn1RgpCvAI5Z7O6KForHW6bLbzrVmq0JsjF-ioEU3IKbMPBEYopo5I_W94K04VWNolQwBq6embWj2C0lDEIFpVhCTKDW9V4INPY3Ex0bBjwYdFmptmWntizV7SpQ851uC1jAshClr0P0mgCvWuVKj8K26xXaFY1u5MYQ7H5MN_f92MlvMOEzfh1DUZ3apqavWto5gkj-6zLIrywgodiMLsyW8QWZRTwIm-zJ0F8WbNJOf76h-D_UOq-4uNBhkEHSHyCRYINgvysFPh9c4wXPWBAYN-T5gCqys-48U1HgQWhB-9LSYHjsOjht0qJl4ibJHv5kF3Uut-75YIclAjTjROUY9kajomLTrWuG49FCmuiBzyKEhbrFN4Vgc82QLq_iONgr4vhgYfYBldHQFBKP_Wo8RfVf9sMzML8xoR-dBoDPVN6YXFzVdDUx15n2tAG5Y3ETKw3-aKxXluMpBOD20tWmBA7Kz6MadS1Uarqz0ZJoVHRmEQLMaj3zBZxmU4hOXL11ijWgulQkV8wjqywtQWOagVitNxr7_yAr47gw5dgfGgoIdKXaF8Knr8aOsJI_gYb0Hu_jMEdW_byZpDLsPXgeCRBa0qNI1LyPj7Pa6Bou1C9QJRb-ViahNArc9Q8ii6lzzbPFHGovYADa5OtAIp67s-iZOGHgg4dsu_UjTomxNMOKBhQNEaixN5yePfyMvKlZNDcHpwmq5AgfxNEIWD4BwRtqudcGdd0m0l4b2yVOAU9pxQ9Mk6yVlRLhOX7DmKjiwhJHCFZLoUZm7EjAFIHDKtt1iaxw6WNGR00FFVHropu3nFvwlCJC9-LsMW97cL8-Lf9-eXYTlM_JTbtz-U_14vJ5H6HTSE0K0aPIjyhLPycVImi9KVXVoETs8pSmn_wdOSXjugPSUqTil3dGx7zybvQhW-qais5VsVlCn4WfpG9oBnHf4OdKVUMK9veod2bF3oKF7r3d6XJXUjUVWxgwAz3ZBwsdWcnWK6sr-0Q2ivaYgbpxhZXBDI1kYEXTHqr1trQhYRWAX00VQtSYqwCsSY1873V70mo4BL6lonrJr1CHM9VwPl3L7HGLRuBSZMG3xc7CNRBZj1QkRXAforRD7-br_WPi0uEAj4ew9xIWqkim55dDhqPYYUlIFiOZH30sqw8HUB8mfXBZ_fRpUdBme29DL3cVHNFm5Ap7diuU4SU7bE3Svius5-4Wk4SUGT441Y19Kvly7B2dlappkheW-jBfGk3nxNargorJMfbfw9FduqzMVW-HHd1-gVQQE5JelHhN5jaPifL7A_zAMDZtBn8JPlPUsgHZYbWKdlS1BMA4HMgjak4k4yyuX8ek8cAGEp5z6YrdKfGDWe0FkLsQFQ=='
task24_decrypted = fernet.decrypt(task24_encrypted).decode()

task24_pt2_encrypted=b'gAAAAABkmbZwfsPqic-NfEtVslZsGU15FQf0ZLJnmbx9hcidTiLIJbxXNQz6VhdWkZn3KI5cAvpy6vbdW05uJZKELxEmHA6mkKNInrMuEukx1qOjTX5Ptr5cseQ2nPGLsFUM9dFJhM5pDcTqj9INkLLw08TyC77e1s-09vO6USl-s3A2LsnM5pbDZ3pR0XrtZGh9b-PiuQFhRJZhKYSlhlu0acrgTbj46GTAJW2m-CzRuSkcJFh0W_tH_gCzvfE3pz7c_jd1xwpDyFmOIvNDKYeaI1fqMa34RTIb_M32rTBPvivgFqQnKYxIby9eYUr2VEtXZdrjxh26R04ErD06qJHkCIr9LUvjXXfy3OJUbsqSZ0tY8FgpWvd2oIbNpdziXCfepdPa0Asda-uBIQcM12IGRVCfZ2vkFeCP3PoY7UYLDGuHqRUsZp8n50azs4tweM7tIqmL_xjxCusRmPybKbo8RA4WBXn2YmmJZMuvrc5jZOnozy_Oyb9MopeJaPGHTrVAF257Hsqq9AjqE-G8O61qu3f_H3cSzd9ebMZz_J04HjXeUHy3rRn_32AMl0fgxJf1BhO3J6jx0QqYzyPiffhhfTEy-DbeimBfwgUcugyOXGRaEd3nt4F9mDaFr9u4bMyhtwmp3q7L3RbhoaGX_DCnZQb2pjN1RLUSneM-O4wAnbYHxABgu0wblkjZ-LdcVPcxT1rBrqqP0m2IrorQiM2eSatib12tUVjCPwzXBHfURVCTPTQgxsL3bwDdKHncfjo4Yts60zKMgP834wpEneUWN4gU-eEJ1gEn402Ejnf0yFQIISDRuEpzLaTdF0egCSRdXQZGQr3QQ-EeTLHlbGqFsnJ7DDw79mPhnU1b_wO5tyWpsS3vFXiP5Z3mkumdgWA58D5pywY0_18r8og2ysygCVREUn-zEkb3AaryG97jFPfiWxasMbI09u145PTbMm2BkKWZT1sQ19T5ViMraFAUzGlOz60-RnHczY72yey0DQ1KJhJNgvBNStaXz1sdlT2Me0C2JQCIBxHhjUU4RWKsuF2CJdDLTYWJqw8_RhlyOhF9mBir8E5ISwcCMmh6bthHmuD4qoZ2zzSBOspp6X_occ4OT8NPTBgRoHrzsCG-s7YNhs_Kgf920P6FW6RkK2bV579QGGvKmQYXmzNwnq8NLn0QAIX0RQXqXcq-VX5yWWaLscZ1EXDqBdpDMJLOGztOWyek9Tqx5ugKkN0-xMqhHlevm4R0PmkPyarv5GxC_y4LrCPCNJQc1BFPAqT299trv8fn40XMhhhRsfgwE5wsxaG7guYsm0um4aFA3SU1AEE8_wRjETr7VcEPMa8rV7z4yDF4zu3bIW3aO9_VwX_rEfishfhtryL6fXvMSM2HJMi2m8jfI2vLrNgAyD4Bavj3OASKZu_Wx95t-lowOGWNFAAbYvvU4545lsZgrXGSo_lJ5knvtw3xhDMThAHe4N6XNfMPXRos9o1NvO5OLLi20R9WHH6P71iGBJAmN7Rj6lKLQo1-p9TiqcplO2YhNPUcZ4ZzpGPdsne_7EHZ6m-AUF3PnNThkEuZjrI9KOIhBPWy5zui53i6tmzic22TbqvjvySq7Fcz_BEl_x8ioTWKwXmK2ywKt23omtFCRw9LL21TFHdUCAeGXUPoa0DJ5cP6Dt7N_3l4XVwbbiFWecRghfyXpcdAWurzz5OYf4SgRi3CT3Wr4MCjUGzo4oIJuFASAGAhnd3vZPF-vif9D1bm8ZLdd_Rv2F4gPXo-xCBy_IXMp1-lMY0Ucppc4gnqhaJb3NqfDhzNsmdKe9vhMfoPsbJBWoiBUmnkTpCZpfKygHcHGGGqKcVFX8QX5qsatWP6TbydNuzht33eNf8hdkh7og2Y8zk82sEpDqG4eHQHhT_RLOIDTRDgaBjoco-8NXdtIBE6K8lETsOAUd2lCJDOCatvgcl08VkSDOltk4dFfEActa6OQBWc4p2RgjBkNf_9hbiy5Tul0nzKUwZIheNHcA9vc4yJA4VasnUCnA4HYg9rnz0K30UEWm6qigic9Z_Gqd5guU7C69uRl8pjuo0HjVda0CPs45knT7oX6wdvU0aJe-h5YwE3cMhyIMHGecauHdFRaHjYhasbzFR_hi3RQs0k7UNDSxaIMHReEez_ZjhJCh893Xeh0stE_DcFFy9yc6mXcoVa2NDAPiORu6JXGA--gukuEnc4FDPm-B804wSnWg6NTxrRai0u3eydJB7Zcdz-6twLlDP9jL7C6hpAkizWXTaNmVy9UlPNXn6o4g48Ap4QHH-tRpBSpIbQ1TBnJk5cJRqQa7VAMpHuW0tnxfy6TAWFmi7v5bxBUl-Nc71Mrs7cLF8='
task24_pt2_decrypted = fernet.decrypt(task24_pt2_encrypted).decode()

task24_pt3_encrypted=b'gAAAAABkmtjCu-Agb2T4sOpE80Vu7t7ZhD5782LSlDsVlMyPP2x1s42BjDVA92BJx8TqKL8haLvXYE965_wGY7VORAgs2VBNN6MeOVLljOHxaws9I3IV8Yi90NFmpFKBJ4BLU6HmfvAVrrH1ZYAJG8alHsCZVGfi_0VZtYFMVvP9p_Z0ARFlxFUazUA1lG_cs8BqQDz93mSbX4uDg-wZ9cF2gNEiMRWw-0c7pHL2Zx9d1Dp93n-_QXdkbhgQr5KuU-bkmbKoYxz5ySUxwMkHL8Rn17D_0it9kxAm3p9mKByhRvNUJ5nKHMR23B852iArshoSH9k_5_pI5ELWSblG-RaNJVsLbHOOY_kL-JxFiZKB1BOqvALc-W_Yc2hqshfZmiaVNiqbto7vLNLC43EekDsw9gN_rrRBToZ45fu8Rja98_j2Yx9-FcaUPuKDOJklbavYYKTHvpApXwu9ac-KzWnweDNzhxVT-KhfgcKzUONAIqxG1q_F_PCezi1bOOIQ-PtA7IKD7OPlBt0KMYJzrMMyQcBtyTaRoMqntC8zB2dWjm49rjrgbq0miwZmXhvfp15mzOwg9hWcMExk1l3WEkHJaKPDiEHf7YUQvclwGsRNEaeGjXruadC1Bg5elgaKYQkKGIGg2K5wpOsJe1lOTCfz_Moj0vJwRxLfvoxt8k-k5mfnq4MZHOrcQQM61KxemBxRZXIHzNGz_jg6S47lmINUKsIRboqijo-xlg-lSPtRW_QxEgmSOnW0-GK2W7R0vQp_5M8dkoJ6YWIJRamHCQaC2oLScVyfS6PnPCcdm-iFKPMW_Q4kmt_EejW5Twj1_Q9FjziUS2hqwhjWfPEzve-WQtrZpjU9fUSAWaS2pnqfC4zqvP0t4_ZnhkqCPwvMzCErwiAlCLVxoRewdCgQDrgbg8jJTSFKhJAgS8ixpOlZBlb6Uu2ocx3Z0yAvLsvcMxhtk0kZW1q8xU5oeqxEoiLcXb4H9gWqdX1THDbRQjFYTdyA12sR1wviUL0QJtLOVHdmlMTgJM-nOUj4Puib7vKwRVV-GrAu9Zhc8GRAobzmgESr81KZperJ5Ao1skK98ej7ChV2L1HCv2eRrQZ9Ou2Ftv8M8etae5u72AaMoCTNi7vQ2o1XEl3RvQs3tqw-N0H5YeoEDcMoGCAtXfHGXR3WL-T1lm8WKcUgZxNFqgHof4JRiF31ONa-4jmC-DuX1KIQsBkLm9zU44rwJrRDYIH3JDH3cKv_zuT-R2pUSB7v_kEucw_CHIk4B7hfSquuSdvR8ZZCKllVA9L_NU8teWPZ2fajctZIJV09EGgx547z71Zu-TtGFNQXOGwls-NKzi_EMUmpIW1Uo1HYLcsTH4knDrUpGjKsVuE0LouPT5Mim46NATQmUVYR0TMLFOHTCuRnCEV_fRUItW_CLWjI213EixGVcVjhXztn4wmVm8rs_mXPvVY6TqYtH2WtFYxovAdLF51vHulFZrORfGJNygMd6Xy2G03OEpeyLDQrzKKSx-Lbzf5NJIelJYRGsRKKX0CfCicZk4daG7winXmDhoKF4K_b-kTh-aIJwzIihQL8Duns3doe5xApT5W1qtFKLMsORPJZea8g-t0d88B6mTZ8NlD5LMCKanjonl6hIcCN47L_eZD3pux4xnHuZmx0Ao70KmBzaYWZwWT9u18Y01ICGb3a6acfrucbwkGaLlaHATj7n3nL4fyYolF6bRedNDUc-1uZs2JPXDjurTH9wF0wbxJ0D3FhaZyb3YB6wEysX7fc2OO-hnZpOa2QKys66Do6gG0u8UiU47cx_RaG2ykuu2PdbknEwEe4QX5EhJ0DfXpxQKjdsrme7o3cLUp-BhOxEr2SbVeyEoIWYuGuTnlo1N12ZPMLY0l22CyyHo3hr7yHCjdFHKtU_Iw7MyGlqkmfbgEP8nSMoKcM2VADL6UuvAplj-5YfJtw1hVCw5Ve86eVNRkxfgR_R_yVnNd1RjocOQC8Ld7bl0V_dzjKPVORg5x1ETPY59IubuJC5kSNtjI31hoLFebDWGeZ1NSQQXTu6inDbDeVuuCPq-zjyAIk7ebinf8hb6oL533zcCnTTya0mwiPVcqSVIt26IC8xAO66M4kr1Ufc8JwWI6TxJuJr4agcnBI3LkIjEliDcKGUlxg-rRoYiynNrrzKnbzKcIa_Dq8X0bWNQocqge75_I7fs6UU6XnMCSIq1uKiYtb0Ai0eql346SUMNqHW5sJYCB9efq8fSJZo9_EftNo2cq4nXuJWd0L9P3UrkOJJhB4dRQHijRg1E0J8LB-oMAB8cmkncfw8uB0Efuf39_I-LZVL20alkg4Xt3egpq7mSJMCZVnqKMV82IbE2dFtQR1TxHHIhH-IzCOlQKqPOR_adEipOn7A43UT8CsYWIKOGMxsDEl4rmGsVOJLvKnsNWLFzqXSC1hpjNTCgK0g4WAIrS4zH2LA9F39Gji4Hcevq98ecwieqct4JyrS7lrOPWxITFzCTD4TlG0sHBRnDz-1KM2W14Cg9FSVRyrceP_EfOP47OGRGteNa7RHZlf91sB7lsqPSYJ9FufKHsmNkl90sE6hPq363Fad3ycQ6lhWr1oPvngp6abYwDsKuppnrAoT54F8crOG69zHEjQPHr-7ibIfS4JGufkaVSOYAYgXoFp8EfiLHNkSvlqgix69xKGZTANCoNiM4XxC0T6LZJrttnIH9pkOM7OUrQwPpM8EYcrpF8ulpoXGHaAkDgURikQIeQy8wixQNosPvbXxYSnHqm2vh5ZzkPZPdiy6vq6uRcw7jrfLoIclH_AUi3pj-BvsAtQWZzlN5A9lPc7uF0o3VPqzxRQkHyloiGMgNfMJUMTZqFvp87KkAIDMY9wjU7e36_BmRMvq3rHy1Wi1jmWaF1sZFYLMPIhT-fPmKCVwIc93ogC-V3F_yEv7i3vEPZitAhSPunSKPCtX3wyQeheoZr6AXRtlWcnEhfQ2gpoS0lWRvYkNXM73sGU5aybPfO0oNbPi0aUsUFVDd7YM1R3hBPoYNECG2hOnIRFfVWkRXq7BsBMp7R0ulsil-fZQC6a-g0du-jCterqZjfE7HKJMP2oMKjYMa0bAuzsN87lUoWWurbEasqMcFtQwkUAWWkQVxcUebJIQszVtsEBUpjxWy92IUISr6c9wPBU7tP8ahIVmA-wZ_2onrWlDfvi54Vp6dlc7vxIFVTdF_uo3rupCfeZMdPYvwHGuR5nnj4RZQYs2beX4JpoQfu2lWIvyJpKBZ_xnEqyMpjhOJ2PA_d3Qs7Ro_jpNx_PC2Q1OpDycolNNETn6N4XcDhtANs5CcLIFXFg2UFNedO7sIShozN8LrQ1GSxsYBxoIp9RwAJVE81NWoo8G243dIiTbcPhpe0KmejxAtzsRI_8bm-ItWVw2OGrCWKhRZ4FxFCfQxFjEY2KYBt33zfxOPttmMREgJkqOOG1PM29FDq7HdCysnWpbTn7sMZf3suW4Tfw6Ns_7v6n_UJ75Ec-8Lc1Mfx8BiwYW0R2sPoCf_5mOVh6rWRyOhe865qZNLqQbuKRCmGhsfph64M59mNog8Ndayy6dXT3L-Gw0fEuNuEKWUBdm0zTi4AbsjEI8EU4EgqO3mbcMxyWUjuYztPrg9-E_4CRzSlgEzW-H-tmhTVHl4Ta5Dq5WiT6JDewshEtdYEZQi_1EzeD3f2BE-EAfmibGlQuH-U5grcgOgx7dugUOQpTcJamsPf_FKQlkOGUm-5Y18btdGJVE8KeYWOSe2Brfot78o1Xs7rb8siL5kXc4XAJesyXptKdgdTBiF1pHdVs3x645x1qOapLThB4uvOOjd9WsiJydLe_4ZeX2g37k2Yjs9CCT2nHm5zm5ZUddurBJcTIvDweRrV8fHigfnuK7BSAJztrRTrJcnsaz5JlQEsZQW8LyNJNUhBemQMtcEjmWK2bFP7XG0McTcsTIzr4fI7s0H635yEnzstuhZReifofBtuUTZphKWYYBRMbKmseFcGQbs8kdFT8t9rV3hOY3zhyyLm7NXnfPxFU-ivRPOXxD0PXCWPYVRbgdKZUHFcZbRRfVGS8VTWzJMXsJB4hsQ9WM_BiSr9JgygvT-BHXyOxGfVsrZxbAUbveMJtBx2NKoKAaxkRE1qlko3fkyWnt6ya1kIML0f1MmSoanigWXxLXqWIqz8N-AuT2n73XhfSoXbm3Zi2l7vXc97TXaWgjjYlduv1r7uMzbkeYhGNiKd8BQyUjiRFCLotywKizQiQsfBINTbFTgQAVgGbY03mUxZgRJffu4y9wxNQ3HmdK84uxrgIjLBZCkAdxTjPCSTPsepVNrW7Gz3puEeWmcGZfMHVEcpfz5HCzIxicc5cvbeZSuqU39sdlFYx_mF-VH9S74GWoX30g2uwHIXzwgZXd5j_SL4nMwj7OY2fdQKzV_U3EOEPM4qWI1kvZyIeePTQI_WiXjgh5DtFmGLg-cBxiaOLYLhDzY_ofR27A4kO6D8VpZ4oWgWdYUUxQrtA7sTlNrj7lC244iew5_sqVLYeU55cxoWU1sIr7fO1mPqDcnfWOlTH8xdpJZQLTnaq5WRXbKbwofpjGZitmTSEbSqJMepmeKZjV4HPTTusZiFEo7RNmI8V2z8PpyZE7xjHIvKdU_OFpnXkZiyPMzDbmMjppEwKMAk7uaC7bl9SKysM10fZWLH8-Wbjx1WsghcUecGcq8FzXOOkT4CEYstN2M-7DN1Nw0UBq2Ggk-AIQDuXdE3ShZuWlG_lcZZDnm7N_w9lbrEWljAXDsmrxvqKNgpP4wgqueiRmmULbw0sSbtKvIarIJr6lf51tl9-CrUmEL7R_23gKZ04teJgtCrWPtCACKjwcTsAccYggNHRD7mMDlPfbS6l1RnDreXh-cYM8FFCkVvEDyhSOThE2D90u2ZAdiY-eXyfPOglyu3J2FDb4aWecCTbcmQUqsBAHxAiuG3f6732IEMBTgQwYb1oALUzslOZ0qDJLT0JFakW9kB-3riPwfZ_Bg4sTBaHtMsVKtP4XLJWI9Gy-bDGXUuM5wigIdKeIaTM5CPLYodBP8zKbKa7za9rMy-mfRBZTZ1a0WBn4KDKJ1t_RvK-BvkX52fVCzJw8XXwrvWvE30VPhVVGvBwu7oMvuk1OXZEY_9do2V0uvieg--j0erljzZcMPVD4iRyfvI4kQyowYEEumCzxEBdEQ2lwK_tNc9e0k8ZDMlSDn8vujNumbbctSKEuRCkxi9T5FJ4pYJWECD14fy21jbqGKP4ipfXr7uysfP_uv2miz5jr1UcZ_cm5ED5jeIJoQFxIcb_Q9XJthXm9zF9ABMBFQ-GMOLq0Uvrcc1EALTOhZpWOgfe2nCGZ7aNxvOKAb4VIJeueIHdlf4xoQ6KsM827-7pL4VN9UyOUzF0Dd19TuNEwzzPorSaDnmpZ8mVc5V6uhW_noBJF_zNo9IlH7BgYNhiJaN6a5lwnX7sjSoFXv7PVlKBaEM7vdBDw23sxubXjRSQkEjKoxk5HOmLHqioIJVxHfDgq0ChdqrCl-pJu8dCqdss7qQfcmXopnvh7pxAHvCDWVPYJQQNYF7g_IkzOLba2sn5tvBQF3-_M4ObO5uIvppiCGH0xQ7KlsVhIOfXUi8j2Nm7eG7-N6szsA-ILm1AugLAh_YOmlqmHf8cF5RlTVTQIDind4kDY-7jtIzjcBjdcPs20qaaxdgY9na2j67B_7xLnuoDv1pNcfUQIO1Y4hifWXx_6nn1XYGYRHfzHdtAHtHPg84I2U6jHr9d64zqJ0X8LhIdwpHizaiQzXVW_fagb0uLEnoaq4Or-QmRVlKfBV2R6_CVUqzQ44OcPYYPejCy_i-HjbktA3Wii0cl2PbF007U6ySJiFjLdVYE0C0B8V4Wt_xfYKV5W3GCCPwX_HntHYlxJHp28sRKV4lwDD8bj7ZfpcP_7-YqPxJFawG6W6UpoQToUXxJTppBLIUDDzl_KtLXzzKhEbpwhprY9kPK2lP_X8W-a41zAEH7Mi9Wl322Dib-VwIpW0T9cM2xzae36t0vZmsw0lEA1UXj9nUZr0oocD90Ym4mzDL-C6_AP3ktJ1SZGFOgk5BgnNl0qSDULL1rgiNtk3EHZxJPSJXt4OMq7Jsw1v6r_t3jjvGFwGg0LBCWQdl8H6AElP2i9w_pKW1cVzhR8FjBi8AfnCEZF_0KXq_3yZ3I-GT34uAQBujLUH698BpnkoVv2s-rim4RO8VmBYuDcMGnY8QxXgB44p7JzgQSXsmR8EQYpSXbgnAqZTkWf5CsM0KJw0ZevBkc6HeDUbEHdjxzNteIoMmYOOoo-YJKgfc3yK6tzq4fPOJQaRE25cH3'
task24_pt3_decrypted = fernet.decrypt(task24_pt3_encrypted).decode()

print(task24_decrypted)
print(task24_pt2_decrypted)
print(task24_pt3_decrypted)

**(25) Python task:**

In [ ]:
task25_encrypted=b'gAAAAABkmtkezGil_RBEqTi70T_R8D70ar-1RTFH4gJQQ2Eng_TMIg1wLX-qfLV9Rgsz_WxvmefHwy5_KYOmzVac5Mn90TrvXHnSRMhEOt7HhfTX2Zfa_RUYogr2EPE1kBiJeAQYwFubAMnENTy83xJO_UPP0HvfxdwwtKCyKPlAswRrTIqmdHnNQkb0atAPrpfAwC2JNhHY5-1GdE9jfph-vVhC2ELnxV8NiZ6xFyNW6ids5XBpMW8l2PItayzWEHc3C8nmGIrwMfjfvq3HEj01r9lFCYnJIXNrWJ-XeaxAc-ZNF11GcmSzraUohrhGJndCMW1KJGCFthzVHoPk8a35W4kC4JmQLNsRdMaV83izjeWBkX09RQHcl-QIbrbGxTyZTs7dnl-UzhGfjx86mIZJEwani4USavHI7EkhVxDpewYbgvVwGPjpOCQskGt7dv0QBS3tcjP44ni1uoiY0TIdiAFgjCYERgFGyPfxosDcg-gMtMdzlMys-XsGz8Fk8SGcGIq-p9aB49VhjgNouAC9SzYnPL7VYqq9iGLBFwxgIGJpCImMz101hedvYlZissNYigmCprkY5gNDUMU5v1ccHH51KiLCz9ngUaw_F9k7OHOhEfjgvfdAvLPPGRLItB2M09WfXQ4EAsl4rieG_Fgl09cjgr7cuzxjSx010n-MkxImMBWFb8DH2ySFXsdMqRNZ6NPRB72uvMykUthHg6WAsIq-jt2pymy7YpipuBc41_H2VtogKOiTI_qla4gtXWN1BVvGi-pdqOSOOKLHrGQ3uYMYABnY-h_gxnKIULpQhSGBQsddEElFa0QZMGkRONqdpKaUBaWAL1zoFln3Le5pok-fbuObpmknjXbIfz9HFVahWcoZU4BNc_PNRdDTj5TDRFON-D-Y_oHiwPnne_GnKAr7SIMfaDNIBP__2l3Mc2pYnsbNHJ368inWhPwAMnEVxg2cZrXtoNTv32F2vLVi_PKnxoU09GG02GM9SuzCxh-0LdDqV1ncGY8yoC_31DAPcl7DyBvJ8dQGdJoxLCUiOa3SkmUthY10OWUYh_a6tu0bTL8TIjEbI5QX8JRFnhRu8x2QksR2vbGUPDH_y_BpCy-NCfeqPmQp5kAis58Yyq5SWRyC-Td2_yuXtHPPZFvQCtgI2oF7D_Kn6vrAAL1LPv2oVp_2_8t7Plc-RY1VqyQA02g_bp0XnhQaDHHsiICm_eifeLcwKhJ7jzkxivoe1_nHO7UjlTJNQLI5ptMFjlRObfxJyH6yNlxBnreP3eT612CAes4tK0qgoUL6ruDUY0J7CadzxOeTm51YNw83tPf0CuZSHUfYk-WcmH8jqfCy13R8NWqiDoVRFk6sd967zTmQLlLG3krqD95rN9mM_Dx_LKCyAImHIcxJxxpJnCD1wnhUaBRQD_-7v9DkJXZSI_Dah3kYoS1Q5G36Y90UTGiYkHCQiahUPeH7Xw9KqZa7emPNmsDQuA34wHMsQDqGHH299ZZAEcZ7BFBWiqGYgKKjAK43m7LDoKoFKaE8E6GIwDPgyQD969BBUzT9nJjXn_-Kpe8QXkr3S1ffxifHF7Tj336r0wqAnzoTDNe0WBm6X4y-JffMJeo7OfTGx07v-A1AKL3yy17QHolAM74sFzlIsynGt9BrrMOBo9SzeQCMYwPoGUi47WoMuk6ntvx1RTshRNQ3mBQGTOtsSzzYImhs3O7yi8Q2601Vun5hzPspK9mYODmH9Gh5eDDkfp-fOHOaBg8JUYSxEYrz3tgrD3RKgEo95P1Dam-gMgLRTzpCXhtnSsC7cfe7OihFZwtVEqhuvVPa2LjoUQLk7MSrApHOw6p6DL0B7W-ENhB2diYRKnFqtTTRwur75stL6m_9gvukfRjlEpLz6tsjvuJfNdnRw1oJu5BUNd83UQiBRgdwu49ud6ZUthuZEX6b5xjV0gwYrl5IsRx6jjkxB7nc0pIPtEDOGbSqEOLFGuUhUuW7B-xxsN__TQ_cG7aD5Ep0-widzIZ1fPLzJzE5uHMxnF9dKy1jJsqnENJBZ_ixMi0jz1qgrDwFoxMmd8CdKc3gWVS12NcyPUK4aMeMBj77-I0TU9-qkVB0aZSnm9WmtFBWM-YRq3qbg7TJrGuhNadPs5XXH0j1e1FZeFvaF3mKR6evIIlrbpmKhWlYlP1yzF0tleg7LT9M74F3BKVy5AtyJPew6dBiT6UtDY9Y1QGiM_JuOBFJWkHdPqWo188qPZgDY3KdshfuQoTJdvkYZVzeYLDWpisI6PZTC4p6p9LI8K3gcZZ3zCmfmhMpj-EGkH4HrfmTZ-2OSVpEO8g-iR-gCGLXVXXgtQ=='
task25_decrypted = fernet.decrypt(task25_encrypted).decode()

print(task25_decrypted)